In [1]:
from pyomo.environ import *

In [2]:
def solve_pyomo(para):
    p_ei  = para['pei']
    p_er  = para['per']
    p_ea  = 1-p_ei-p_er
    p_ai  = p_ei
    p_ar  = p_er
    p_irr = 1/8
    alpha  = 0.6
    beta_1 = para['beta1']
    beta_2 = para['beta2']
    cost_tr = 0.0977
    p_id  = 1.1/1000
    p_idd = 0.1221/1000
    p_srr = 0.7/3
    S = 100000
    E = 20
    A = 0
    I = 0
    R = 0
    D = 0
    N = S+E+A+I+R+D
    K = 50
    w_max  = 1.0
    tr_max = 1.0
    v_max = 0.2/14
    M = 3.6e3
    
    m = ConcreteModel()
    m.k = RangeSet(0, K)
    m.t = RangeSet(0, K-1)
    m.N = N
    m.tri = Var(m.k, bounds=(0, 1)) 
    m.wi = Var(m.k, bounds=(0, 1))  
    m.v = Var(m.k, bounds=(0, 1))
    m.aq = Var(m.k, bounds=(0, 1))
    m.iq = Var(m.k, bounds=(0, 1))
    m.x_s = Var(m.k, domain = NonNegativeReals)
    m.x_e = Var(m.k, domain = NonNegativeReals)
    m.x_a = Var(m.k, domain = NonNegativeReals)
    m.x_i = Var(m.k, domain = NonNegativeReals)
    m.x_r = Var(m.k, domain = NonNegativeReals)
    m.x_d = Var(m.k, domain = NonNegativeReals)
    
    m.xs_c = Constraint(m.t, rule=lambda m, k: m.x_s[k+1] == m.x_s[k]
                    -(beta_1*(1-m.iq[k])*m.x_i[k]+beta_2*(m.x_e[k]+(1-m.aq[k])*m.x_a[k]))
                    *(1-m.v[k]*v_max)*m.x_s[k]/(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_i[k]+m.x_r[k])
                    -p_srr*m.v[k]*v_max*m.x_s[k])

    m.xe_c = Constraint(m.t, rule=lambda m, k: m.x_e[k+1] == m.x_e[k]
                        +alpha*(beta_1*(1-m.iq[k])*m.x_i[k]+beta_2*(m.x_e[k]+(1-m.aq[k])*m.x_a[k]))
                        *(1-m.v[k]*v_max)*m.x_s[k]/(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_i[k]+m.x_r[k])
                        -p_ei*m.x_e[k]
                        -p_er*m.x_e[k]
                        -m.wi[k]*p_ea*m.x_e[k])
                     
    m.xa_c = Constraint(m.t, rule=lambda m, k: m.x_a[k+1] == m.x_a[k]
                        +m.wi[k]*p_ea*m.x_e[k]
                        -p_ai*m.x_a[k]
                        -p_ar*m.x_a[k])
                                        
    m.xi_c = Constraint(m.t, rule=lambda m, k: m.x_i[k+1] == m.x_i[k]
                        +(1-alpha)*(beta_1*(1-m.iq[k])*m.x_i[k]+beta_2*(m.x_e[k]+(1-m.aq[k])*m.x_a[k]))
                        *(1-m.v[k]*v_max)*m.x_s[k]/(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_i[k]+m.x_r[k])
                        +p_ei*m.x_e[k]
                        +p_ai*m.x_a[k]
                        -m.tri[k]*p_irr*m.x_i[k]
                        -(1-m.tri[k])*m.x_i[k]*p_id
                        -m.tri[k]*m.x_i[k]*p_idd)
                                                                                       
    m.xr_c = Constraint(m.t, rule=lambda m, k: m.x_r[k+1] == m.x_r[k]
                        +m.tri[k]*p_irr*m.x_i[k]
                        +p_er*m.x_e[k]
                        +p_ar*m.x_a[k]
                        +p_srr*m.v[k]*v_max*m.x_s[k])

    m.xd_c = Constraint(m.t, rule=lambda m, k: m.x_d[k+1] == m.x_d[k]
                        +(1-m.tri[k])*m.x_i[k]*p_id
                        +m.tri[k]*m.x_i[k]*p_idd)
    
    m.pc = ConstraintList()
    m.pc.add(m.x_s[0]==S)
    m.pc.add(m.x_e[0]==E)
    m.pc.add(m.x_a[0]==A)
    m.pc.add(m.x_i[0]==I)
    m.pc.add(m.x_r[0]==R)
    m.pc.add(m.x_d[0]==D)

    gamma = 1
    m.sumcost = sum(m.wi[k]*(0.000369*(m.wi[k]*(m.x_s[k]+m.x_a[k]+m.x_e[k]+m.x_r[k])/m.N)
                         *(m.wi[k]*(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_r[k])/m.N) + 0.001057)
                         *(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_r[k])
                    +0.0977*m.tri[k]*m.x_i[k]+m.x_s[k]
                    *m.v[k]*v_max*0.07 + m.x_i[k]*m.iq[k]*0.02 + m.x_a[k]*m.aq[k]*0.02 for k in m.t) 
    m.budget_c = Constraint(expr = m.sumcost <= M)

    m.suminfected = sum(2.6 * (beta_1*(1-m.iq[k])*m.x_i[k]+beta_2*(m.x_e[k]+(1-m.aq[k])*m.x_a[k]))
                        *(1-m.v[k]*v_max)*m.x_s[k]/(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_i[k]+m.x_r[k])
                        +(m.x_d[k+1] - m.x_d[k]) * 60 for k in m.t)
    m.obj = Objective(expr = m.suminfected, sense = minimize)
    solver = SolverFactory('ipopt')
    solver.options['max_iter']= 100000
    try:
        solver.solve(m,tee=True)
    except:
        print()

In [3]:
import pickle
interval = 1
para_truth = pickle.load(open('para_truth' + str(round(interval * 100)), 'rb'))

In [4]:
solve_pyomo(para_truth[0])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 302r 7.7964596e+04 7.89e+04 9.99e+02   4.9 0.00e+00   2.6 0.00e+00 4.62e-07R  5
 303r 7.7963272e+04 7.89e+04 1.19e+04   4.9 2.78e+07    -  4.42e-05 6.44e-08f  1
 304r 8.0110425e+04 7.83e+04 2.09e+04   2.8 2.16e+05    -  6.58e-03 2.71e-03f  1
 305r 8.2147434e+04 7.78e+04 2.07e+04   2.8 3.27e+05    -  1.62e-03 1.64e-03f  1
 306r 8.2311491e+04 7.78e+04 2.70e+04   2.8 2.56e+05    -  2.32e-03 1.62e-04f  1
 307r 9.0175834e+04 7.58e+04 1.26e+04   2.8 2.58e+05    -  3.67e-03 7.59e-03f  1
 308r 9.1088524e+04 7.56e+04 1.52e+04   2.1 4.36e+05    -  1.98e-03 5.13e-04f  1
 309r 9.1090037e+04 7.56e+04 8.74e+05   2.1 2.25e+01   2.0 9.33e-01 1.75e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 310r 9.1036922e+04 7.56e+04 2.19e+05   2.1 3.99e+01   1.5 8.01e-01 1.00e+00f  1
 311r 9.1048897e+04 7.56e+04 1.89e+05   2.1 9.35e+01   1.0 1.00e+00 3.69e-01f  1
 312r 9.0973834e+04 7.56e+04 1.72e+05   2.1 3.01e+02   0.6 1.05e-01 8.05e-02f  1
 313r 9.0885075e+04 7.56e+04

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 600r 3.7973053e+05 4.71e+03 9.99e+02   3.7 0.00e+00  -1.1 0.00e+00 2.59e-07R  6
 601r 3.7973435e+05 4.71e+03 1.01e+03   3.7 8.26e+07    -  2.17e-06 3.74e-08f  1
 602r 3.7981206e+05 4.69e+03 1.42e+05   3.0 1.32e+06    -  1.19e-02 1.70e-05f  1
 603r 3.9882469e+05 4.19e+02 1.40e+05   2.3 7.37e+05    -  6.50e-03 6.35e-03f  1
 604  3.9882552e+05 4.19e+02 1.51e+04  -1.0 4.19e+02  -0.7 1.86e-03 2.41e-04h  1
 605  3.9882632e+05 4.19e+02 1.51e+04  -1.0 4.19e+02  -1.2 3.57e-03 3.05e-04h  1
 606  3.9882798e+05 4.18e+02 1.51e+04  -1.0 4.32e+02  -0.8 1.69e-03 6.77e-04h  1
 607  3.9882802e+05 4.18e+02 1.51e+04  -1.0 9.22e+02  -1.2 6.36e-04 9.11e-05h  1
 608  3.9882812e+05 4.18e+02 1.50e+04  -1.0 1.23e+03  -0.8 3.11e-04 9.98e-05h  1
 609  3.9882810e+05 4.18e+02 1.50e+04  -1.0 1.35e+03  -1.3 2.05e-03 8.38e-05h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 610  3.9882810e+05 4.18e+02

 709  3.5866892e+05 4.77e+00 2.73e+03  -1.0 7.56e+02  -3.5 2.77e-01 4.89e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 710  3.5828625e+05 4.54e+00 2.74e+03  -1.0 2.00e+03  -4.0 7.58e-02 5.04e-02f  1
 711  3.5821526e+05 4.41e+00 5.11e+03  -1.0 6.90e+02  -3.6 4.40e-01 2.71e-02f  1
 712  3.5774536e+05 4.09e+00 4.65e+03  -1.0 1.33e+03  -4.0 6.20e-02 7.71e-02f  1
 713  3.5757795e+05 3.78e+00 4.66e+03  -1.0 4.69e+02  -3.6 1.56e-01 7.53e-02f  1
 714  3.5723084e+05 3.57e+00 4.90e+03  -1.0 1.36e+03  -4.1 1.84e-01 5.74e-02f  1
 715  3.5698698e+05 3.18e+00 5.26e+03  -1.0 4.88e+02  -3.7 3.44e-01 1.11e-01f  1
 716  3.5649411e+05 3.60e+00 5.71e+03  -1.0 1.41e+03  -4.1 3.40e-01 7.74e-02f  1
 717  3.5608273e+05 3.19e+00 6.60e+03  -1.0 5.08e+02  -3.7 9.09e-01 1.77e-01f  1
 718  3.5531857e+05 3.00e+00 5.90e+03  -1.0 1.54e+03  -4.2 2.64e-01 1.10e-01f  1
 719  3.5513677e+05 2.80e+00 5.44e+03  -1.0 5.48e+02  -3.8 9.90e-01 7.23e-02f  1
iter    objective    inf_pr 

 814  3.3985098e+05 1.26e-01 4.04e+02  -3.8 4.03e+02  -3.9 1.83e-01 6.67e-02f  1
 815  3.3985091e+05 1.26e-01 4.04e+02  -3.8 1.34e+03  -4.4 1.02e-05 4.00e-05f  1
 816  3.3979285e+05 1.56e-01 4.27e+02  -3.8 4.53e+02  -4.0 1.70e-01 6.93e-02f  1
 817  3.3979263e+05 1.56e-01 4.30e+02  -3.8 1.69e+03  -4.5 9.49e-03 1.17e-04f  1
 818  3.3977877e+05 1.55e-01 4.18e+02  -3.8 5.18e+02  -4.0 5.19e-05 1.52e-02f  1
 819  3.3977871e+05 1.55e-01 4.18e+02  -3.8 1.73e+03  -4.5 2.05e-03 3.48e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 820  3.3974289e+05 1.77e-01 4.03e+02  -3.8 6.48e+02  -4.1 3.89e-03 5.18e-02f  1
 821  3.3969820e+05 1.80e-01 3.95e+02  -3.8 1.73e+03  -4.6 5.43e-05 1.50e-02f  1
 822  3.3964211e+05 1.97e-01 4.63e+02  -3.8 6.45e+02  -4.1 2.48e-01 4.86e-02f  1
 823  3.3964202e+05 1.97e-01 4.63e+02  -3.8 2.44e+02  -3.7 4.65e-05 2.23e-04f  1
 824  3.3950364e+05 3.14e-01 4.29e+02  -3.8 7.24e+02  -4.2 1.59e-01 1.08e-01f  1
 825  3.3950361e+05 3.14e-01

...

2621  5.2013323e+02 2.32e+00 1.97e+04  -3.8 1.64e+03  -7.0 1.16e-05 7.59e-03f  1
2622  5.1912520e+02 2.32e+00 9.22e+03  -3.8 1.51e+03  -7.5 7.84e-02 1.08e-03f  1
2623  5.1162408e+02 2.32e+00 1.00e+04  -3.8 1.58e+03  -7.1 1.28e-05 7.04e-03f  1
2624  4.9967643e+02 2.35e+00 1.11e+04  -3.8 3.33e+03  -7.6 2.23e-03 1.33e-02f  1
2625  4.8923192e+02 2.38e+00 2.57e+03  -3.8 1.80e+03  -7.1 8.49e-02 1.23e-02f  1
2626  4.8538867e+02 2.37e+00 2.96e+03  -3.8 1.51e+03  -6.7 1.31e-05 3.86e-03f  1
2627  4.8517074e+02 2.37e+00 2.96e+03  -3.8 6.06e+03  -7.2 1.92e-04 5.07e-05f  1
2628  4.8513653e+02 2.37e+00 2.96e+03  -3.8 1.51e+03  -6.8 1.31e-05 3.48e-05f  1
2629  4.8444733e+02 2.37e+00 2.94e+03  -3.8 2.14e+04  -4.5 3.27e-05 4.63e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2630  4.8444590e+02 2.37e+00 2.94e+03  -3.8 7.09e+02  -3.2 4.17e-06 3.43e-06f  1
2631  4.8375724e+02 2.37e+00 2.80e+03  -3.8 5.10e+02  -3.7 1.31e-03 1.95e-03f  1
2632  4.8375422e+02 2.37e+00

2724  2.5645924e+02 1.72e-02 2.65e+04  -3.8 2.04e+00  -1.1 5.17e-02 4.71e-02f  1
2725  2.5615261e+02 1.08e-02 2.62e+04  -3.8 1.49e+01  -1.6 2.30e-03 2.29e-02f  1
2726  2.5602053e+02 9.44e-03 2.48e+04  -3.8 1.50e+00  -1.1 1.53e-01 6.33e-02f  1
2727  2.5594597e+02 8.26e-03 2.51e+04  -3.8 5.36e-01  -0.7 6.98e-01 1.07e-01f  1
2728  2.5543098e+02 8.09e-03 1.94e+04  -3.8 1.63e+00  -1.2 1.87e-01 2.26e-01f  1
2729  2.5490082e+02 5.68e-03 5.38e+03  -3.8 6.05e-01  -0.8 1.00e+00 7.47e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2730  2.5459379e+02 6.61e-03 4.60e+03  -3.8 1.80e+00  -1.2 4.24e-01 1.61e-01f  1
2731  2.5411667e+02 1.04e-02 4.14e+03  -3.8 4.85e+00  -1.7 1.11e-01 8.31e-02f  1
2732  2.5405244e+02 1.01e-02 3.95e+03  -3.8 1.55e+00  -1.3 5.38e-01 3.76e-02f  1
2733  2.5361291e+02 1.64e-02 3.63e+03  -3.8 5.71e+00  -1.8 1.04e-01 7.67e-02f  1
2734  2.5348540e+02 1.76e-02 3.57e+03  -3.8 2.28e+02  -2.3 1.96e-03 9.93e-04f  1
2735  2.5334476e+02 1.71e-02

2838  1.2179296e+02 2.94e+00 4.80e+04  -5.7 8.69e+02  -7.6 4.94e-01 5.85e-02f  1
2839  1.2123322e+02 2.91e+00 5.15e+04  -5.7 2.09e+03  -8.1 6.28e-02 1.36e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2840  1.1980056e+02 2.86e+00 8.07e+04  -5.7 6.75e+02  -7.7 4.38e-01 3.58e-02f  1
2841  1.1981066e+02 2.84e+00 8.08e+04  -5.7 4.09e-01   0.2 3.20e-03 6.96e-03h  1
2842  1.1980715e+02 2.84e+00 7.88e+04  -5.7 4.26e-01  -0.3 4.05e-02 2.50e-03f  1
2843  1.1981144e+02 2.82e+00 7.79e+04  -5.7 1.68e-01  -0.8 1.12e-02 6.23e-03h  1
2844  1.1981152e+02 2.82e+00 3.15e+03  -5.7 1.47e-01  -1.2 1.00e+00 1.28e-04h 13
2845  1.1990338e+02 2.43e+00 2.66e+03  -5.7 3.05e-01  -1.7 1.85e-01 1.38e-01h  1
2846  1.2042731e+02 1.38e-02 6.91e+02  -5.7 7.91e-01  -2.2 5.09e-01 1.00e+00h  1
2847  1.2035697e+02 9.22e-03 5.38e+02  -5.7 2.29e+00  -2.7 1.00e+00 3.43e-01h  1
2848  1.2013968e+02 7.51e-03 3.14e+02  -5.7 6.80e+00  -3.2 3.29e-01 3.52e-01f  1
2849  1.1983374e+02 9.31e-03

2952  8.1831577e+01 1.21e+01 1.78e+04  -5.7 4.43e-02   0.7 1.00e+00 3.71e-02h  1
2953  8.1829708e+01 1.21e+01 1.74e+04  -5.7 5.53e+02    -  2.31e-02 4.92e-03h  1
2954  8.1832839e+01 1.18e+01 4.22e+03  -5.7 1.17e-01   0.2 1.00e+00 2.39e-02h  1
2955  8.1826617e+01 1.11e+01 4.24e+03  -5.7 4.42e+01    -  3.53e-04 5.48e-02h  1
2956  8.1826787e+01 1.11e+01 2.78e+03  -5.7 1.77e-01  -0.2 4.46e-01 1.41e-03h  1
2957  8.1826793e+01 1.11e+01 4.91e+03  -5.7 1.08e-01  -0.7 1.00e+00 4.21e-05h  1
2958  8.1800419e+01 8.36e+00 4.34e+03  -5.7 4.14e+01    -  1.27e-01 2.48e-01h  1
2959  8.1800422e+01 8.36e+00 1.10e+03  -5.7 5.35e-02  -1.2 9.21e-01 3.23e-05h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2960  8.1800416e+01 8.36e+00 3.86e+02  -5.7 3.17e+01    -  3.37e-01 6.58e-05h  1
2961  8.1800432e+01 8.36e+00 6.59e+02  -5.7 9.08e-02  -1.7 9.73e-02 1.73e-04h  1
2962  8.1800451e+01 8.36e+00 8.30e+02  -5.7 8.73e-02  -2.1 6.84e-02 2.02e-04h  1
2963  8.1800470e+01 8.36e+00

3078  7.8704170e+01 2.06e-03 4.66e+02  -5.7 2.27e-02  -1.5 1.00e+00 3.03e-01f  1
3079  7.8704132e+01 5.82e-11 2.57e-02  -5.7 4.82e-03  -2.0 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
3080  7.8698900e+01 3.52e-02 8.31e+02  -5.7 3.43e+04    -  9.34e-04 5.49e-04f  1
3081  7.8698861e+01 3.29e-02 7.60e+02  -5.7 5.97e+00    -  8.82e-02 6.17e-02h  1
3082  7.8674520e+01 1.96e+00 1.06e+03  -5.7 2.46e+03    -  2.75e-02 3.92e-02f  1
3083  7.8623781e+01 2.08e+01 8.93e+03  -5.7 5.35e+02    -  1.54e-01 4.30e-01h  1
3084  7.8627630e+01 1.85e+01 7.77e+03  -5.7 1.87e+02    -  2.68e-01 1.19e-01h  1
3085  7.8623249e+01 1.75e+01 1.20e+03  -5.7 6.46e+02    -  2.96e-01 9.13e-02h  1
3086  7.8623245e+01 1.75e+01 1.19e+03  -5.7 2.22e+02    -  1.75e-03 3.04e-04h  1
3087  7.8623391e+01 1.74e+01 3.36e+03  -5.7 1.48e+02    -  3.79e-01 8.55e-03h  1
3088  7.8623305e+01 1.73e+01 4.99e+03  -5.7 8.03e+02    -  8.16e-02 3.53e-03h  1
3089  7.8623602e+01 1.15e+01

In [5]:
solve_pyomo(para_truth[1])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 153  2.5650108e+05 3.88e+04 5.26e+03  -1.0 3.88e+04  -1.0 3.32e-04 1.16e-04h  1
 154  2.5656621e+05 3.88e+04 5.26e+03  -1.0 3.88e+04  -1.5 3.19e-04 2.01e-04h  1
 155  2.5663017e+05 3.88e+04 5.26e+03  -1.0 3.88e+04  -2.0 3.78e-04 2.18e-04h  1
 156  2.5663483e+05 3.88e+04 5.27e+03  -1.0 3.88e+04  -1.6 8.42e-04 1.58e-05h  1
 157  2.5663866e+05 3.88e+04 5.30e+03  -1.0 3.88e+04  -1.1 8.25e-04 1.30e-05h  1
 158  2.5672490e+05 3.87e+04 5.30e+03  -1.0 3.88e+04  -1.6 5.58e-04 3.23e-04h  1
 159  2.5673419e+05 3.87e+04 5.30e+03  -1.0 3.87e+04  -2.1 2.42e-04 3.74e-05h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 160  2.5673954e+05 3.87e+04 5.31e+03  -1.0 3.87e+04  -1.7 7.00e-04 2.12e-05h  1
 161  2.5675587e+05 3.87e+04 5.31e+03  -1.0 6.22e+04  -2.1 7.86e-05 6.52e-05h  1
 162  2.5676313e+05 3.87e+04 5.32e+03  -1.0 3.87e+04  -1.7 5.12e-04 3.02e-05h  1
 163  2.5682935e+05 3.87e+04 5.32e+03  -1.0 3.87e+04  -2.2 2.56e-04 2.94e-04h  1
 164r 2.5682935e+05 3.87e+04

 251  3.6103848e+05 1.32e+04 8.88e+04  -1.0 3.44e+04    -  7.49e-02 3.45e-03h  1
 252  3.6186410e+05 1.28e+04 8.86e+04  -1.0 3.36e+04    -  5.19e-04 2.56e-02h  1
 253  3.6193585e+05 1.28e+04 9.21e+04  -1.0 3.34e+04  -3.7 2.11e-02 1.93e-03h  1
 254  3.6201084e+05 1.28e+04 9.21e+04  -1.0 3.32e+04    -  2.38e-04 2.35e-03h  1
 255  3.6203493e+05 1.28e+04 9.23e+04  -1.0 3.35e+04  -4.2 4.99e-02 6.81e-04h  1
 256  3.6230365e+05 1.26e+04 9.21e+04  -1.0 3.41e+04    -  1.91e-03 1.17e-02h  1
 257  3.6235755e+05 1.26e+04 9.21e+04  -1.0 3.30e+04    -  4.41e-04 1.77e-03h  1
 258  3.6237637e+05 1.26e+04 9.50e+04  -1.0 3.32e+04  -3.8 2.02e-02 5.24e-04h  1
 259  3.6260569e+05 1.25e+04 9.50e+04  -1.0 3.30e+04    -  7.55e-04 7.57e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 260  3.6262175e+05 1.25e+04 9.45e+04  -1.0 3.32e+04  -4.2 4.13e-02 4.80e-04h  1
 261  3.6313576e+05 1.23e+04 9.42e+04  -1.0 3.30e+04    -  3.42e-03 1.79e-02h  1
 262  3.6338247e+05 1.21e+04

 389  3.4552270e+05 1.41e+03 1.08e+04  -1.0 2.25e+04  -4.9 5.88e-02 1.01e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 390  3.4541008e+05 1.40e+03 9.56e+03  -1.0 2.25e+04  -5.4 7.08e-02 9.21e-03f  1
 391  3.4417742e+05 1.28e+03 9.65e+03  -1.0 2.34e+04  -4.9 6.72e-03 8.22e-02f  1
 392  3.4414701e+05 1.28e+03 9.44e+03  -1.0 3.21e+04  -5.4 1.12e-02 5.87e-04f  1
 393  3.4110426e+05 1.09e+03 9.66e+03  -1.0 2.29e+04  -5.0 1.73e-02 1.51e-01f  1
 394  3.4060977e+05 1.08e+03 9.43e+03  -1.0 5.91e+04  -5.5 1.61e-02 8.04e-03f  1
 395  3.4059314e+05 1.08e+03 8.61e+03  -1.0 2.20e+04  -5.1 6.21e-02 1.18e-03f  1
 396  3.3933980e+05 9.87e+02 1.97e+03  -1.0 2.10e+04  -5.5 5.29e-01 8.43e-02f  1
 397  3.3903036e+05 9.61e+02 1.93e+03  -1.0 2.06e+04  -5.1 2.61e-02 2.71e-02f  1
 398  3.3681534e+05 8.32e+02 1.32e+03  -1.0 2.05e+04  -5.6 3.55e-02 1.34e-01f  1
 399  3.3667616e+05 8.23e+02 1.22e+03  -1.0 1.96e+04  -5.2 6.47e-02 1.09e-02f  1
iter    objective    inf_pr 

 514  3.0949857e+05 8.06e+00 5.66e+03  -1.0 1.47e+03  -3.2 3.81e-02 1.74e-03f  1
 515  3.0944814e+05 7.65e+00 5.39e+03  -1.0 1.68e+03  -3.7 1.03e-01 5.18e-02f  1
 516  3.0940113e+05 7.16e+00 4.92e+03  -1.0 1.52e+03  -3.3 5.54e-02 6.64e-02f  1
 517  3.0939475e+05 7.11e+00 4.64e+03  -1.0 1.63e+03  -3.8 5.96e-01 6.07e-03f  1
 518  3.0935412e+05 6.78e+00 4.64e+03  -1.0 1.58e+03  -3.3 6.48e-02 4.82e-02f  1
 519  3.0927861e+05 6.37e+00 3.80e+03  -1.0 1.64e+03  -3.8 4.66e-01 6.47e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 520  3.0926660e+05 6.26e+00 3.68e+03  -1.0 1.39e+03  -3.4 1.04e-03 1.76e-02f  1
 521  3.0912321e+05 5.54e+00 2.91e+03  -1.0 1.49e+03  -3.9 7.01e-01 1.27e-01f  1
 522  3.0910748e+05 5.31e+00 2.76e+03  -1.0 1.09e+03  -3.5 3.05e-02 4.26e-02f  1
 523  3.0890555e+05 4.63e+00 2.11e+03  -1.0 1.39e+03  -3.9 7.94e-01 1.44e-01f  1
 524  3.0890561e+05 4.63e+00 2.80e+03  -1.0 4.89e+00   1.0 1.54e-02 3.03e-04h  1
 525  3.0890635e+05 4.61e+00

...

1138  5.5325074e+04 7.38e+01 8.00e+03  -3.8 1.08e+05  -5.0 2.66e-03 1.74e-03f  1
1139  5.5324669e+04 7.38e+01 8.00e+03  -3.8 2.88e+07  -5.5 2.66e-09 6.03e-09f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1140  5.4765532e+04 7.37e+01 8.08e+03  -3.8 1.42e+05  -5.1 2.13e-03 1.41e-03f  1
1141  5.4765340e+04 7.37e+01 8.08e+03  -3.8 2.48e+04  -4.6 1.53e-06 3.01e-06f  1
1142  5.4169283e+04 7.36e+01 8.17e+03  -3.8 1.98e+05  -5.1 1.63e-03 1.08e-03f  1
1143  5.4169094e+04 7.36e+01 8.17e+03  -3.8 2.84e+04  -4.7 1.50e-06 2.60e-06f  1
1144  5.3533687e+04 7.36e+01 8.27e+03  -3.8 3.05e+05  -5.2 1.13e-03 7.42e-04f  1
1145  5.3533495e+04 7.36e+01 8.27e+03  -3.8 3.28e+04  -4.7 1.47e-06 2.28e-06f  1
1146  5.2856419e+04 7.35e+01 8.38e+03  -3.8 5.98e+05  -5.2 6.14e-04 4.02e-04f  1
1147  5.2856222e+04 7.35e+01 8.38e+03  -3.8 3.82e+04  -4.8 1.43e-06 2.02e-06f  1
1148  5.2855928e+04 7.35e+01 8.38e+03  -3.8 4.58e+06  -5.3 3.50e-09 2.26e-08f  1
1149  5.2066392e+04 7.31e+01

1249  5.6018777e+03 1.11e+01 9.95e+02  -3.8 2.68e+03  -3.8 8.08e-03 2.41e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1250  5.1671014e+03 9.08e+00 1.08e+03  -3.8 8.29e+02  -3.4 6.64e-05 1.28e-01f  1
1251  5.1670836e+03 9.08e+00 1.08e+03  -3.8 3.61e+02  -2.9 1.87e-03 1.28e-05f  1
1252  5.0419815e+03 8.55e+00 1.09e+03  -3.8 1.03e+03  -3.4 1.25e-03 3.00e-02f  1
1253  5.0418602e+03 8.55e+00 1.02e+03  -3.8 4.01e+02  -3.0 3.99e-02 7.75e-05f  1
1254  4.9309392e+03 8.11e+00 1.04e+03  -3.8 1.25e+03  -3.5 4.70e-04 2.26e-02f  1
1255  4.9309255e+03 8.11e+00 1.04e+03  -3.8 1.29e+04  -3.9 4.99e-06 2.99e-07f  1
1256  4.8484905e+03 7.92e+00 1.03e+03  -3.8 1.03e+03  -3.5 9.85e-03 1.98e-02f  1
1257  4.8445804e+03 7.90e+00 1.03e+03  -3.8 4.53e+02  -3.1 3.63e-04 2.10e-03f  1
1258  4.6524218e+03 7.63e+00 1.05e+03  -3.8 1.58e+03  -3.6 1.42e-04 3.12e-02f  1
1259  4.6519447e+03 7.63e+00 9.35e+02  -3.8 4.98e+02  -3.1 7.29e-02 2.28e-04f  1
iter    objective    inf_pr 

1359  1.4678723e+03 5.72e-01 4.11e+02  -3.8 2.57e+03  -5.7 4.51e-02 3.56e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1360  1.3522855e+03 8.16e-01 3.44e+02  -3.8 2.61e+03  -6.2 7.57e-06 1.94e-02f  1
1361  1.3521293e+03 8.16e-01 4.50e+02  -3.8 3.53e+03  -6.7 3.35e-02 3.38e-05f  1
1362  1.2892016e+03 1.04e+00 4.24e+02  -3.8 1.51e+04  -7.1 7.87e-03 1.33e-02f  1
1363  1.1542527e+03 1.59e+00 3.27e+02  -3.8 3.47e+03  -6.7 4.88e-03 3.24e-02f  1
1364  1.1482355e+03 1.59e+00 3.26e+02  -3.8 2.29e+04  -7.2 9.80e-07 1.65e-04f  1
1365  1.1481151e+03 1.59e+00 4.65e+02  -3.8 3.26e+03  -6.8 4.59e-02 3.41e-05f  1
1366  1.1086277e+03 1.66e+00 4.27e+02  -3.8 1.24e+04  -7.2 7.77e-06 9.93e-03f  1
1367  1.1068976e+03 1.66e+00 4.29e+02  -3.8 1.60e+04  -6.8 2.71e-04 3.24e-05f  1
1368  1.1062514e+03 1.66e+00 4.28e+02  -3.8 1.13e+04  -7.3 7.66e-06 1.71e-04f  1
1369  1.1007940e+03 1.66e+00 4.27e+02  -3.8 5.04e+04  -6.9 9.14e-06 3.93e-05f  1
iter    objective    inf_pr 

1472  4.6662122e+02 1.07e-01 5.71e+03  -3.8 3.06e+01  -2.5 4.77e-02 7.21e-02f  1
1473  4.6586866e+02 9.26e-02 3.79e+03  -3.8 8.84e+00  -2.1 2.51e-01 9.78e-02f  1
1474  4.6497503e+02 8.83e-02 3.74e+03  -3.8 2.14e+01  -2.6 2.84e-02 4.84e-02f  1
1475  4.6496318e+02 8.83e-02 3.73e+03  -3.8 6.53e+01  -3.1 3.75e-04 2.25e-04f  1
1476  4.6423715e+02 8.77e-02 3.67e+03  -3.8 2.00e+02  -3.5 1.48e-02 4.48e-03f  1
1477  4.6168249e+02 5.90e-02 3.66e+03  -3.8 1.79e+03  -3.1 1.66e-06 1.83e-03f  1
1478  4.6167887e+02 5.90e-02 3.62e+03  -3.8 3.78e+02  -3.6 1.02e-02 1.28e-05f  1
1479  4.5961981e+02 5.89e-02 3.62e+03  -3.8 1.35e+03  -3.2 1.58e-06 2.11e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1480  4.5961521e+02 5.89e-02 3.57e+03  -3.8 3.37e+02  -3.6 1.28e-02 1.99e-05f  1
1481  4.5713253e+02 5.86e-02 3.56e+03  -3.8 9.15e+02  -3.2 2.15e-06 4.10e-03f  1
1482  4.5712882e+02 5.86e-02 3.54e+03  -3.8 5.46e+02  -3.7 9.30e-03 1.08e-05f  1
1483  4.5543581e+02 5.84e-02

1607  3.6297646e+01 1.06e+01 9.03e+02  -3.8 1.15e+00   0.3 1.60e-05 7.42e-03h  1
1608  3.6299071e+01 1.06e+01 1.25e+03  -3.8 1.74e+00   1.7 7.43e-02 5.09e-03h  1
1609  3.6217640e+01 9.52e+00 1.45e+03  -3.8 3.36e+02    -  2.02e-04 1.04e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1610  3.6212754e+01 8.85e+00 3.03e+02  -3.8 5.47e-02   1.2 9.77e-01 7.04e-02h  1
1611  3.6203971e+01 7.94e+00 2.40e+02  -3.8 2.23e-01   0.7 4.75e-01 1.05e-01h  1
1612  3.6190949e+01 7.11e+00 1.94e+02  -3.8 2.48e-01   0.2 1.00e+00 1.05e-01h  1
1613  3.5689774e+01 9.43e+00 6.88e+02  -3.8 6.65e+02    -  2.33e-02 2.43e-01h  1
1614  3.5081513e+01 2.06e+01 1.03e+03  -3.8 3.16e+02    -  2.83e-01 8.50e-01h  1
1615  3.5315559e+01 8.71e+00 5.40e+02  -3.8 1.01e+02    -  2.83e-01 8.03e-01h  1
1616  3.5314461e+01 8.62e+00 6.64e+02  -3.8 3.81e-01  -0.3 1.00e+00 9.86e-03h  1
1617  3.5066306e+01 7.43e+00 4.81e+02  -3.8 2.83e+02    -  3.08e-01 6.14e-01h  1
1618  3.5074671e+01 4.23e+00

In [6]:
solve_pyomo(para_truth[2])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 316  2.5040794e+05 3.18e+04 4.64e+03  -1.0 3.18e+04  -2.9 9.65e-05 8.97e-05h  1
 317  2.5041797e+05 3.18e+04 4.64e+03  -1.0 3.18e+04  -3.4 1.82e-04 7.54e-05h  1
 318  2.5043723e+05 3.18e+04 4.64e+03  -1.0 3.18e+04  -2.1 7.55e-05 6.10e-05h  1
 319r 2.5043723e+05 3.18e+04 9.99e+02   4.5 0.00e+00  -2.6 0.00e+00 3.05e-07R  6
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 320r 2.5043728e+05 3.18e+04 1.69e+03   4.5 5.68e+07    -  1.91e-05 2.52e-08f  1
 321r 2.5333166e+05 3.09e+04 8.93e+03   3.1 1.22e+06    -  7.27e-03 7.07e-04f  1
 322r 2.9065920e+05 2.07e+04 8.60e+03   2.4 1.43e+06    -  2.04e-02 7.17e-03f  1
 323  2.9067282e+05 2.07e+04 5.34e+03  -1.0 2.07e+04  -3.0 7.35e-04 8.35e-05h  1
 324  2.9068906e+05 2.07e+04 5.34e+03  -1.0 2.07e+04  -1.7 2.07e-04 6.20e-05h  1
 325  2.9070394e+05 2.07e+04 5.34e+03  -1.0 2.07e+04  -2.2 1.34e-04 6.66e-05h  1
 326  2.9073560e+05 2.07e+04 5.34e+03  -1.0 2.07e+04  -1.8 8.02e-04 1.27e-04h  1
 327  2.9074803e+05 2.07e+04

 431  3.2515842e+05 4.70e+03 3.23e+05  -1.0 2.50e+04  -4.5 5.48e-02 1.03e-02f  1
 432  3.2464588e+05 4.49e+03 3.17e+05  -1.0 2.49e+04  -4.1 1.08e-02 4.42e-02f  1
 433  3.2421232e+05 4.33e+03 3.15e+05  -1.0 2.47e+04  -4.6 1.02e-02 3.46e-02f  1
 434  3.2395671e+05 4.24e+03 3.01e+05  -1.0 2.45e+04  -4.2 4.87e-02 2.21e-02f  1
 435  3.2391879e+05 4.23e+03 2.98e+05  -1.0 2.45e+04  -4.6 8.63e-03 1.83e-03f  1
 436  3.2362848e+05 4.19e+03 2.98e+05  -1.0 2.46e+04  -5.1 1.92e-04 1.06e-02f  1
 437  3.2341955e+05 4.15e+03 2.67e+05  -1.0 2.45e+04  -4.7 9.24e-02 9.76e-03f  1
 438  3.2298048e+05 4.03e+03 2.66e+05  -1.0 2.44e+04  -4.3 2.65e-03 2.87e-02f  1
 439  3.2294584e+05 4.02e+03 2.63e+05  -1.0 2.82e+04  -4.7 1.01e-02 1.90e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 440  3.2224152e+05 3.83e+03 2.54e+05  -1.0 2.43e+04  -4.3 3.71e-02 4.58e-02f  1
 441  3.2221579e+05 3.83e+03 2.50e+05  -1.0 2.43e+04  -4.8 1.33e-02 9.62e-04f  1
 442  3.2197199e+05 3.78e+03

 543  2.6286742e+05 2.38e+02 1.60e+04  -1.0 1.23e+04  -3.8 9.03e-02 4.95e-03f  1
 544  2.5782974e+05 1.84e+02 4.97e+03  -1.0 1.28e+04  -4.3 3.83e-01 2.27e-01f  1
 545  2.5750992e+05 1.77e+02 5.52e+03  -1.0 1.08e+04  -3.8 1.24e-01 3.58e-02f  1
 546  2.5391060e+05 1.50e+02 7.70e+03  -1.0 1.13e+04  -4.3 7.48e-02 1.56e-01f  1
 547  2.5059414e+05 1.41e+02 1.51e+04  -1.0 1.83e+04  -4.8 2.82e-03 5.62e-02f  1
 548  2.5015351e+05 1.28e+02 1.31e+04  -1.0 9.18e+03  -3.5 2.47e-02 9.36e-02f  1
 549  2.5009764e+05 1.27e+02 8.89e+03  -1.0 8.95e+03  -3.9 3.04e-02 4.99e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 550  2.4433046e+05 1.02e+02 1.29e+04  -1.0 9.72e+03  -4.4 9.53e-02 2.03e-01f  1
 551  2.4178642e+05 7.85e+01 1.81e+04  -1.0 7.76e+03  -4.0 5.68e-02 2.26e-01f  1
 552  2.4177847e+05 7.84e+01 1.50e+04  -1.0 6.35e+03  -3.6 1.51e-01 1.66e-03f  1
 553  2.3846441e+05 5.92e+01 1.35e+04  -1.0 6.90e+03  -4.0 1.97e-01 2.46e-01f  1
 554  2.3837861e+05 5.90e+01

 657  2.1092483e+05 6.83e-02 5.04e+01  -3.8 2.51e+02  -3.1 3.21e-01 2.93e-01f  1
 658  2.1008935e+05 8.32e-01 3.00e+01  -3.8 7.60e+02  -3.6 4.34e-01 4.25e-01f  1
 659  2.1008921e+05 8.32e-01 2.81e+02  -3.8 2.29e+03  -4.0 1.63e-01 2.35e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 660  2.0885963e+05 5.01e+00 4.84e+02  -3.8 6.06e+03  -4.5 8.87e-02 7.84e-02f  1
 661  2.0885943e+05 5.01e+00 4.83e+02  -3.8 2.23e+03  -4.1 2.12e-05 3.38e-05f  1
 662  2.0881147e+05 4.99e+00 4.79e+02  -3.8 6.41e+03  -4.6 3.04e-03 2.57e-03f  1
 663  2.0878713e+05 4.97e+00 4.65e+02  -3.8 2.22e+03  -4.2 8.04e-03 3.64e-03f  1
 664  2.0827156e+05 1.64e+01 5.41e+02  -3.8 1.09e+04  -4.6 1.15e-02 2.40e-02f  1
 665  2.0827146e+05 1.64e+01 5.19e+02  -3.8 2.48e+03  -4.2 6.36e-03 1.31e-05f  1
 666  2.0807404e+05 1.53e+01 5.79e+02  -3.8 9.19e+02  -3.8 3.27e-02 7.03e-02f  1
 667  2.0746862e+05 1.48e+01 6.43e+02  -3.8 3.34e+03  -4.3 2.28e-05 6.66e-02f  1
 668  2.0746851e+05 1.48e+01

...

1105  3.7185495e+03 5.99e+00 4.34e+02  -3.8 7.83e+04  -5.5 7.62e-10 5.19e-08f  1
1106  3.7185466e+03 5.99e+00 4.34e+02  -3.8 5.05e+04  -5.1 4.84e-09 2.39e-08f  1
1107  3.7185465e+03 5.99e+00 4.34e+02  -3.8 1.86e+04  -5.6 4.94e-08 2.82e-09f  2
1108  3.7185467e+03 5.99e+00 4.36e+02  -3.8 2.11e+05  -6.0 6.75e-05 2.82e-09f  2
1109  3.7184932e+03 5.99e+00 4.42e+02  -3.8 3.78e+04  -6.5 7.86e-04 9.50e-06f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1110  3.6102427e+03 5.97e+00 4.45e+02  -3.8 3.88e+04  -5.2 5.73e-07 1.00e-03f  1
1111  3.6084210e+03 5.97e+00 4.41e+02  -3.8 5.67e+04  -5.7 6.06e-04 1.15e-05f  1
1112  3.5148687e+03 5.96e+00 4.45e+02  -3.8 7.47e+04  -6.2 6.41e-08 4.60e-04f  1
1113  3.4992589e+03 5.96e+00 4.44e+02  -3.8 2.99e+04  -4.8 4.39e-04 2.27e-04f  1
1114  3.4991020e+03 5.96e+00 4.41e+02  -3.8 3.83e+04  -5.3 7.31e-04 1.48e-06f  1
1115  3.4385400e+03 5.95e+00 4.44e+02  -3.8 5.37e+04  -5.8 5.99e-08 4.02e-04f  1
1116  3.4332458e+03 5.95e+00

1217  8.7816839e+02 3.10e+00 2.31e+02  -3.8 7.66e+02  -3.9 5.18e-02 6.53e-02f  1
1218  8.7816369e+02 3.10e+00 2.31e+02  -3.8 2.70e+04  -4.3 5.23e-08 9.48e-08f  1
1219  8.7815066e+02 3.10e+00 2.31e+02  -3.8 1.53e+03  -4.8 2.30e-06 4.63e-06f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1220  8.7815060e+02 3.10e+00 2.31e+02  -3.8 5.63e+03  -5.3 2.40e-09 6.11e-09f  2
1221  8.7815060e+02 3.10e+00 3.05e+03  -3.8 1.43e+00   2.5 9.06e-03 7.75e-08h  2
1222  8.7497967e+02 3.09e+00 3.08e+03  -3.8 1.50e+04    -  9.77e-03 2.04e-03f  1
1223  8.7498442e+02 3.09e+00 3.24e+03  -3.8 4.82e-01   2.1 1.86e-03 9.70e-04h  1
1224  8.7502063e+02 3.06e+00 2.75e+03  -3.8 7.16e-01   1.6 3.23e-03 8.47e-03h  1
1225  8.7541577e+02 2.78e+00 1.33e+03  -3.8 8.61e-01   1.1 1.65e-02 9.42e-02h  1
1226  8.7613886e+02 2.23e+00 6.62e+02  -3.8 5.66e-01   0.6 4.49e-02 1.95e-01h  1
1227  8.7629111e+02 1.97e+00 6.17e+02  -3.8 7.04e-01   0.2 1.85e-01 1.18e-01h  1
1228  8.7610199e+02 1.89e+00

1343  9.6645701e+01 2.02e-05 7.27e+00  -3.8 3.58e-02   0.5 1.00e+00 7.85e-01f  1
1344  9.6590498e+01 7.99e-05 9.78e+00  -3.8 1.08e-01  -0.0 1.00e+00 4.88e-01f  1
1345  9.6440061e+01 5.71e-04 3.79e+00  -3.8 3.20e-01  -0.5 1.00e+00 4.54e-01f  1
1346  9.6343062e+01 5.92e-04 3.52e+00  -3.8 7.39e-01  -1.0 2.68e-01 1.03e-01f  1
1347  9.6056893e+01 1.51e-03 5.96e+00  -3.8 2.32e+00  -1.5 4.01e-01 1.06e-01f  1
1348  9.5530534e+01 4.93e-03 7.67e+00  -3.8 6.55e+00  -1.9 9.54e-02 6.98e-02f  1
1349  9.4761475e+01 1.41e-02 9.37e+00  -3.8 1.80e+01  -2.4 4.64e-02 3.86e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1350  9.3624484e+01 3.38e-02 5.03e+00  -3.8 4.52e+01  -2.9 1.51e-02 2.39e-02f  1
1351  9.3445500e+01 3.43e-02 5.80e+00  -3.8 2.06e+02  -3.4 3.14e-03 1.58e-03f  1
1352  9.2730536e+01 5.96e-02 5.85e+00  -3.8 2.04e+04    -  6.35e-05 1.08e-04f  1
1353  9.2230824e+01 5.94e-02 4.54e+00  -3.8 1.45e+02  -3.8 2.35e-03 3.47e-03f  1
1354  9.1817950e+01 5.85e-02

1481  3.8068247e+01 7.34e-08 3.07e-02  -5.7 9.23e-02  -3.8 1.00e+00 1.00e+00h  1
1482  3.8068238e+01 5.43e-08 1.75e-01  -5.7 1.70e-01  -4.3 1.00e+00 1.00e+00h  1
1483  3.8068235e+01 9.03e-08 5.92e+00  -5.7 5.99e-01  -4.8 1.00e+00 1.71e-01h  2
1484  3.8068166e+01 1.49e-02 5.14e+00  -5.7 1.24e+03    -  9.78e-02 1.65e-01h  1
1485  3.8068197e+01 8.09e-02 1.87e+00  -5.7 1.85e+02    -  7.04e-01 1.00e+00h  1
1486  3.8068170e+01 1.29e-02 8.75e+00  -5.7 1.62e+05    -  1.00e+00 1.45e-01H  1
1487  3.8068148e+01 2.67e-02 8.05e-03  -5.7 8.52e+01    -  1.00e+00 1.00e+00h  1
1488  3.8068151e+01 9.41e-03 1.19e-03  -5.7 7.89e+01    -  1.00e+00 1.00e+00h  1
1489  3.8068150e+01 3.73e-05 1.57e-04  -5.7 1.47e+01    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1490  3.8068150e+01 1.19e-07 3.04e-06  -5.7 1.17e+00    -  1.00e+00 1.00e+00h  1
1491  3.8067995e+01 4.11e-03 1.35e+01  -8.6 2.96e+02    -  4.37e-01 3.63e-01h  1
1492  3.8067938e+01 1.47e+01

1584  3.8067868e+01 1.46e-02 1.95e+01  -8.6 3.98e+02  -8.4 9.46e-02 1.11e-01h  1
1585  3.8067882e+01 1.54e-02 1.81e+01  -8.6 4.65e+02  -8.9 1.04e-01 1.63e-02h  3
1586  3.8067835e+01 1.46e-02 1.66e+01  -8.6 1.06e+03  -9.4 9.93e-02 3.98e-02h  1
1587  3.8067814e+01 2.48e-02 1.33e+01  -8.6 9.32e+02  -9.9 2.52e-01 5.92e-02h  1
1588  3.8067804e+01 2.35e-02 1.24e+01  -8.6 5.30e+02 -10.4 4.18e-01 5.57e-02h  1
1589  3.8067791e+01 2.61e+00 1.06e+01  -8.6 1.86e+02  -9.9 2.13e-02 1.21e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1590  3.8067780e+01 1.30e+00 2.09e+00  -8.6 8.19e-01  -5.9 5.01e-01 2.02e-01h  1
1591  3.8067747e+01 3.27e-03 7.05e+00  -8.6 3.83e-01  -5.5 4.36e-01 7.97e-01h  1
In iteration 1591, 1 Slack too small, adjusting variable bound
1592  3.8067744e+01 2.38e-03 2.21e+01  -8.6 1.03e+00  -5.9 8.88e-02 2.71e-01h  1
1593  3.8067738e+01 1.27e-03 1.26e+01  -8.6 2.26e+00  -6.4 7.54e-01 4.69e-01h  1
1594  3.8067734e+01 8.84e+00 1.04e+01  -8.6 3.

In [7]:
solve_pyomo(para_truth[3])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 305  3.8653260e+04 8.98e+04 1.44e+03  -1.0 8.98e+04   0.1 3.17e-07 5.83e-08f  3
 306  3.8653296e+04 8.98e+04 1.44e+03  -1.0 8.98e+04  -0.4 6.08e-08 9.32e-08f  3
 307r 3.8653296e+04 8.98e+04 1.00e+03   5.0 0.00e+00  -0.9 0.00e+00 2.85e-07R  2
 308r 3.8645108e+04 8.98e+04 1.00e+03   5.0 1.28e+09    -  8.47e-09 5.03e-09f  1
 309r 3.8640723e+04 8.98e+04 1.00e+03   2.9 1.18e+06    -  2.61e-05 5.53e-06f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 310r 3.8662657e+04 8.98e+04 1.00e+03   2.9 9.52e+05    -  1.94e-05 9.93e-05f  1
 311r 3.8720959e+04 8.98e+04 1.71e+03   2.9 4.63e+05    -  2.52e-03 6.96e-05f  1
 312r 3.9277111e+04 8.96e+04 1.77e+03   2.9 3.67e+05    -  5.76e-04 4.84e-04f  1
 313r 3.9958890e+04 8.94e+04 1.90e+03   2.9 3.68e+05    -  8.09e-04 6.48e-04f  1
 314r 4.0790707e+04 8.92e+04 2.33e+03   2.9 3.39e+05    -  1.23e-03 7.96e-04f  1
 315r 4.1821670e+04 8.89e+04 2.92e+03   2.9 4.07e+05    -  1.56e-03 9.71e-04f  1
 316r 4.2761104e+04 8.86e+04

 431  2.1799914e+05 3.12e+02 2.92e+06  -1.0 1.50e+03  -1.5 5.69e-05 4.27e-03h  1
 432  2.1800704e+05 3.11e+02 2.87e+06  -1.0 2.71e+03  -2.0 1.87e-02 1.61e-03h  1
 433  2.1801264e+05 3.11e+02 2.86e+06  -1.0 1.49e+03  -1.5 3.59e-03 1.07e-03h  1
 434  2.1811803e+05 3.04e+02 2.73e+06  -1.0 1.63e+03  -2.0 5.63e-02 2.09e-02h  1
 435  2.1812928e+05 3.04e+02 2.72e+06  -1.0 1.37e+03  -1.6 7.54e-03 2.22e-03h  1
 436  2.1817413e+05 3.01e+02 2.70e+06  -1.0 1.36e+03  -1.2 1.01e-02 8.69e-03h  1
 437  2.1820421e+05 2.99e+02 2.68e+06  -1.0 1.30e+03  -1.6 1.61e-02 5.92e-03h  1
 438  2.1821036e+05 2.99e+02 2.78e+06  -1.0 1.27e+03  -1.2 2.46e-02 1.20e-03h  1
 439  2.1824963e+05 2.97e+02 2.71e+06  -1.0 1.26e+03  -1.7 7.58e-02 7.98e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 440  2.1832480e+05 2.92e+02 2.67e+06  -1.0 1.24e+03  -1.3 1.47e-02 1.52e-02h  1
 441  2.1836781e+05 2.89e+02 2.63e+06  -1.0 1.20e+03  -1.7 2.20e-02 9.05e-03h  1
 442  2.1840585e+05 2.87e+02

 544  2.1723107e+05 2.39e+01 8.75e+04  -1.0 3.84e+02  -3.0 1.35e-01 9.73e-04f  1
 545  2.1723517e+05 2.37e+01 9.00e+04  -1.0 1.42e+03  -3.5 9.85e-03 8.04e-03h  1
 546  2.1696405e+05 1.58e+01 3.64e+05  -1.0 3.95e+02  -3.1 6.30e-04 3.33e-01f  1
 547  2.1695202e+05 1.58e+01 1.15e+05  -1.0 9.45e+02  -3.6 2.69e-01 3.26e-03f  1
 548  2.1695191e+05 1.58e+01 1.13e+05  -1.0 3.38e+02  -3.1 2.17e-02 3.48e-04h  1
 549  2.1681605e+05 1.51e+01 9.08e+04  -1.0 1.03e+03  -3.6 1.04e-03 3.87e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 550  2.1681036e+05 1.51e+01 1.31e+05  -1.0 3.25e+02  -3.2 1.46e-01 3.89e-03f  1
 551  2.1563573e+05 1.14e+01 1.47e+05  -1.0 1.08e+03  -3.7 2.57e-02 2.56e-01f  1
 552  2.1536103e+05 1.12e+01 1.49e+05  -1.0 4.69e+03  -4.1 1.01e-02 1.79e-02f  1
 553  2.1535872e+05 1.12e+01 6.40e+04  -1.0 1.26e+03  -3.7 1.37e-01 5.17e-04f  1
 554  2.1439179e+05 5.51e+00 2.80e+05  -1.0 4.40e+02  -3.3 1.36e-02 5.10e-01f  1
 555  2.1436646e+05 5.48e+00

 658  1.6766099e+05 7.90e-01 1.47e+04  -2.5 1.08e+03  -3.2 2.22e-02 3.10e-02f  1
 659  1.6761247e+05 8.47e-01 1.43e+04  -2.5 3.78e+03  -3.7 2.37e-02 3.74e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 660  1.6745125e+05 1.15e+00 1.38e+04  -2.5 1.22e+03  -3.3 2.83e-02 4.07e-02f  1
 661  1.6738857e+05 1.25e+00 1.33e+04  -2.5 4.46e+03  -3.8 2.10e-02 4.53e-03f  1
 662  1.6730060e+05 1.36e+00 1.27e+04  -2.5 1.39e+03  -3.3 1.20e-01 1.98e-02f  1
 663  1.6715672e+05 2.05e+00 1.20e+04  -2.5 4.97e+03  -3.8 3.00e-02 9.07e-03f  1
 664  1.6700898e+05 2.57e+00 1.07e+04  -2.5 1.39e+03  -3.4 3.05e-01 2.89e-02f  1
 665  1.6686196e+05 2.95e+00 9.73e+03  -2.5 5.19e+02  -3.0 6.41e-03 7.70e-02f  1
 666  1.6686181e+05 2.95e+00 7.63e+03  -2.5 4.89e+01  -1.6 1.16e-01 7.70e-04f  1
 667  1.6681537e+05 2.51e+00 1.15e+04  -2.5 6.55e+01  -2.1 1.00e+00 1.69e-01f  1
 668  1.6670332e+05 2.76e+00 1.00e+04  -2.5 2.33e+02  -2.6 6.48e-03 1.23e-01f  1
 669  1.6670317e+05 2.58e+00

...

1115  3.1031666e+03 2.09e+01 7.82e+07  -2.5 1.27e+01   5.1 3.83e-02 2.17e-04h  1
1116  3.1032187e+03 2.07e+01 7.67e+07  -2.5 1.27e+01   5.6 1.18e-02 1.22e-02h  1
1117  3.1032496e+03 2.05e+01 7.57e+07  -2.5 1.26e+01   5.1 6.12e-03 7.21e-03h  1
1118  3.1033308e+03 2.01e+01 5.89e+07  -2.5 1.25e+01   5.5 6.52e-03 1.91e-02h  1
1119  3.1033355e+03 2.01e+01 8.45e+07  -2.5 1.22e+01   5.0 8.96e-02 1.13e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1120  3.1033742e+03 1.99e+01 7.75e+07  -2.5 1.22e+01   5.5 3.07e-03 9.20e-03h  1
1121  3.1034530e+03 1.95e+01 1.04e+08  -2.5 1.21e+01   5.0 1.52e-01 1.89e-02h  1
1122  3.1034538e+03 1.95e+01 1.08e+08  -2.5 1.19e+01   5.4 5.36e-03 2.16e-04h  1
1123  3.1035100e+03 1.92e+01 1.13e+08  -2.5 1.20e+01   4.9 7.07e-02 1.63e-02h  1
1124  3.1035668e+03 1.89e+01 1.12e+08  -2.5 1.18e+01   5.4 1.83e-02 1.67e-02h  1
1125  3.1035946e+03 1.87e+01 1.14e+08  -2.5 1.16e+01   4.9 5.36e-02 8.10e-03h  1
1126  3.1036284e+03 1.85e+01

1227  2.6716932e+03 1.21e-02 8.46e+03  -3.8 2.77e-02   1.4 8.96e-01 7.19e-01f  1
1228  2.6716225e+03 8.05e-03 1.20e+04  -3.8 8.21e-02   0.9 1.00e+00 3.50e-01f  1
1229  2.6712942e+03 2.74e-03 7.21e+03  -3.8 2.20e-01   0.4 4.36e-01 6.83e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1230  2.6710487e+03 2.24e-03 2.48e+03  -3.8 6.47e-01  -0.1 1.00e+00 1.80e-01f  1
1231  2.6681388e+03 5.17e-04 4.33e+03  -3.8 1.79e+00  -0.5 1.43e-01 7.71e-01f  1
1232  2.6659576e+03 4.38e-04 9.96e+02  -3.8 5.36e+00  -1.0 6.74e-01 1.92e-01f  1
1233  2.6320955e+03 5.86e-03 2.40e+03  -3.8 1.60e+01  -1.5 1.58e-01 1.00e+00f  1
1234  2.6262334e+03 5.70e-03 5.52e+03  -3.8 4.75e+01  -2.0 5.03e-01 5.84e-02f  1
1235  2.6262333e+03 5.70e-03 3.96e+04  -3.8 1.08e-01   2.1 3.29e-02 3.84e-04h  1
1236  2.6262333e+03 5.57e-03 5.14e+04  -3.8 9.16e-03   1.6 1.00e+00 2.16e-02f  3
1237  2.6262025e+03 1.35e-07 7.20e+00  -3.8 2.96e-02   1.1 1.00e+00 1.00e+00h  1
1238  2.6259700e+03 1.36e-07

1349  8.1361637e+02 1.19e+00 6.21e+04  -3.8 3.98e+00  -1.7 2.26e-01 8.10e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1350  8.1373566e+02 1.13e+00 1.53e+04  -3.8 2.44e+00  -1.3 2.74e-01 5.29e-02h  1
1351  8.1373321e+02 1.09e+00 2.95e+03  -3.8 5.89e+00  -1.8 1.45e-01 3.39e-02h  1
1352  8.1367820e+02 1.08e+00 4.11e+03  -3.8 3.12e+01  -2.3 2.46e-02 7.90e-03h  1
1353  8.1312357e+02 1.08e+00 5.59e+03  -3.8 3.86e+02    -  1.34e-02 1.55e-03f  1
1354  8.1312388e+02 1.08e+00 5.59e+03  -3.8 2.65e+03    -  6.78e-08 6.10e-07f  9
1355  8.1312322e+02 1.08e+00 6.71e+03  -3.8 5.50e+01  -2.7 8.91e-03 3.20e-05h  1
1356  7.9107580e+02 1.10e+00 5.89e+03  -3.8 5.18e+02    -  2.56e-03 4.59e-02f  1
1357  7.9107219e+02 1.10e+00 5.22e+03  -3.8 1.79e+01  -2.3 1.20e-02 6.78e-04h  1
1358  7.8633749e+02 1.11e+00 6.51e+03  -3.8 5.14e+02    -  1.62e-04 1.02e-02f  1
1359  7.7274342e+02 1.11e+00 9.84e+03  -3.8 4.89e+02    -  3.81e-03 3.05e-02f  1
iter    objective    inf_pr 

1476  1.5318539e+02 1.27e-05 1.05e-01  -5.7 2.84e-04   1.6 1.00e+00 1.00e+00f  1
1477  1.5318530e+02 1.37e-05 1.34e+00  -5.7 2.88e-04   1.1 1.00e+00 9.88e-01h  1
1478  1.5318522e+02 1.46e-11 2.85e-03  -5.7 6.15e-04   0.7 1.00e+00 1.00e+00f  1
1479  1.5318501e+02 1.46e-11 2.39e-03  -5.7 1.55e-03   0.2 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1480  1.5318460e+02 1.96e-11 2.17e+01  -5.7 3.99e-03  -0.3 1.00e+00 7.79e-01f  1
1481  1.5318391e+02 7.28e-11 3.32e+01  -5.7 1.11e-02  -0.8 1.00e+00 4.76e-01f  1
1482  1.5318140e+02 9.02e-10 1.96e+01  -5.7 3.25e-02  -1.2 1.00e+00 6.01e-01f  1
1483  1.5317614e+02 3.96e-09 1.26e+01  -5.7 9.74e-02  -1.7 1.00e+00 4.23e-01f  1
1484  1.5316417e+02 2.09e-08 3.83e+01  -5.7 2.92e-01  -2.2 1.00e+00 3.23e-01f  1
1485  1.5313962e+02 5.53e-06 1.45e+02  -5.7 8.75e-01  -2.7 1.00e+00 2.22e-01f  1
1486  1.5309357e+02 3.21e-05 3.65e+02  -5.7 2.62e+00  -3.2 7.07e-01 1.40e-01f  1
1487  1.5309358e+02 1.46e-11

1593  9.8321609e+01 1.52e+01 2.09e+04  -5.7 3.57e-02   1.5 1.29e-01 7.49e-02h  1
1594  9.8322829e+01 1.50e+01 1.15e+04  -5.7 6.45e-02   1.0 7.73e-01 1.29e-02h  1
1595  9.8345522e+01 1.17e+01 7.05e+03  -5.7 3.48e-02   1.5 1.94e-01 2.18e-01h  1
1596  9.8345626e+01 1.17e+01 1.85e+04  -5.7 7.82e-02   1.0 8.19e-01 1.45e-03h  1
1597  9.8370435e+01 8.50e+00 2.61e+04  -5.7 3.27e-02   1.4 8.85e-01 2.75e-01h  1
1598  9.8381457e+01 7.26e+00 6.73e+04  -5.7 2.58e-02   1.8 4.83e-01 1.46e-01h  1
1599  9.8382231e+01 7.17e+00 2.89e+04  -5.7 3.29e-02   1.3 1.00e+00 1.27e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1600  9.8403079e+01 4.88e+00 5.88e+04  -5.7 2.25e-02   1.8 1.00e+00 3.19e-01h  1
1601  9.8415607e+01 3.36e+00 5.62e+04  -5.7 3.42e-02   1.3 1.00e+00 3.15e-01h  1
1602  9.8431079e+01 1.75e+00 7.26e+03  -5.7 1.42e-02   1.7 1.00e+00 4.80e-01h  1
1603  9.8434912e+01 8.71e-01 3.83e+04  -5.7 3.11e-02   1.2 1.00e+00 5.19e-01h  1
1604  9.8439343e+01 1.30e-01

In [8]:
solve_pyomo(para_truth[4])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 314  2.6275962e+05 2.84e+04 2.34e+06  -1.0 3.88e+04  -2.7 1.33e-01 3.10e-02h  1
 315  2.6518945e+05 2.75e+04 3.11e+06  -1.0 3.80e+04  -2.3 1.95e-01 3.28e-02h  1
 316  2.6560058e+05 2.73e+04 7.53e+06  -1.0 3.72e+04  -1.8 2.66e-01 5.70e-03h  1
 317  2.6792624e+05 2.64e+04 7.03e+06  -1.0 3.75e+04  -2.3 2.44e-01 3.29e-02h  1
 318  2.6814541e+05 2.64e+04 7.13e+06  -1.0 3.67e+04  -1.9 1.43e-02 3.16e-03h  1
 319  2.6871634e+05 2.61e+04 7.08e+06  -1.0 3.72e+04  -2.4 6.93e-04 8.03e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 320  2.6896130e+05 2.61e+04 7.20e+06  -1.0 3.67e+04  -1.9 1.84e-02 3.46e-03h  1
 321  2.7064378e+05 2.54e+04 6.23e+06  -1.0 3.63e+04  -1.5 1.64e-03 2.46e-02h  1
 322  2.7117688e+05 2.52e+04 6.25e+06  -1.0 3.63e+04  -2.0 1.92e-02 8.03e-03h  1
 323  2.7141707e+05 2.51e+04 6.13e+06  -1.0 3.59e+04  -1.6 3.33e-04 3.68e-03h  1
 324  2.7317895e+05 2.44e+04 5.85e+06  -1.0 3.64e+04  -2.1 2.48e-03 2.78e-02h  1
 325  2.7323908e+05 2.44e+04

 421  3.4861520e+05 9.96e+02 7.01e+04  -1.0 5.58e+03  -2.0 6.09e-03 2.13e-05h  1
 422  3.4861067e+05 9.96e+02 7.04e+04  -1.0 6.86e+03  -2.5 2.81e-03 5.97e-04f  1
 423  3.4860920e+05 9.95e+02 7.04e+04  -1.0 1.13e+04  -3.0 5.09e-04 1.48e-04f  1
 424  3.4860789e+05 9.95e+02 7.07e+04  -1.0 7.33e+03  -2.5 1.77e-03 1.37e-04f  1
 425  3.4860386e+05 9.95e+02 7.07e+04  -1.0 1.43e+04  -3.0 2.60e-04 4.04e-04f  1
 426  3.4860359e+05 9.95e+02 7.31e+04  -1.0 7.77e+03  -2.6 1.08e-02 2.68e-05h  1
 427  3.4859285e+05 9.94e+02 7.33e+04  -1.0 1.06e+04  -3.1 4.30e-03 5.81e-04f  1
 428  3.4859009e+05 9.94e+02 7.35e+04  -1.0 9.34e+03  -2.6 8.21e-04 1.66e-04f  1
 429  3.4858375e+05 9.94e+02 7.36e+04  -1.0 1.29e+04  -3.1 2.83e-03 3.22e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 430  3.4858332e+05 9.94e+02 7.48e+04  -1.0 1.16e+04  -2.7 3.42e-03 3.00e-05h  1
 431  3.4856224e+05 9.93e+02 7.49e+04  -1.0 1.31e+04  -3.2 2.86e-03 1.22e-03f  1
 432  3.4856140e+05 9.92e+02

 535  3.1792706e+05 1.79e+02 3.16e+04  -1.0 2.82e+04  -4.9 2.01e-02 7.33e-03f  1
 536  3.1781528e+05 1.78e+02 3.40e+04  -1.0 1.72e+04  -4.5 6.92e-02 6.31e-03f  1
 537  3.1740480e+05 1.75e+02 3.58e+04  -1.0 1.82e+04  -5.0 5.84e-02 1.73e-02f  1
 538  3.1357791e+05 1.43e+02 2.71e+04  -1.0 1.74e+04  -4.6 1.08e-01 1.78e-01f  1
 539  3.1322977e+05 1.42e+02 2.79e+04  -1.0 2.34e+04  -5.0 2.74e-02 7.03e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 540  3.1133494e+05 1.30e+02 2.99e+04  -1.0 1.68e+04  -4.6 2.20e-01 8.79e-02f  1
 541  3.1131699e+05 1.30e+02 3.02e+04  -1.0 1.67e+04  -5.1 7.78e-03 7.82e-04f  1
 542  3.0909850e+05 1.14e+02 2.84e+04  -1.0 1.55e+04  -4.7 1.91e-01 1.24e-01f  1
 543  3.0909960e+05 1.14e+02 2.86e+04  -1.0 1.02e+04  -1.5 1.42e-04 1.65e-04h  1
 544  3.0910223e+05 1.14e+02 2.15e+04  -1.0 9.50e+03  -2.0 3.85e-03 2.05e-03h  1
 545  3.0910648e+05 1.13e+02 1.49e+04  -1.0 1.12e+04  -2.5 7.50e-03 3.12e-03h  1
 546  3.0911230e+05 1.13e+02

 647  2.7356285e+05 2.10e+00 1.63e+02  -3.8 1.04e+03  -4.2 2.66e-04 1.16e-01f  1
 648  2.7356262e+05 2.10e+00 1.25e+02  -3.8 6.34e+01  -2.9 7.00e-02 1.76e-03f  1
 649  2.7356227e+05 2.10e+00 1.36e+03  -3.8 2.45e+02  -3.3 3.45e-01 8.48e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 650  2.7353247e+05 2.04e+00 1.39e+03  -3.8 4.46e+02  -3.8 1.71e-01 3.09e-02f  1
 651  2.7348940e+05 2.02e+00 1.38e+03  -3.8 1.34e+03  -4.3 2.29e-02 1.50e-02f  1
 652  2.7336467e+05 1.89e+00 1.22e+03  -3.8 5.26e+02  -3.9 1.20e-05 1.19e-01f  1
 653  2.7334706e+05 1.37e+00 8.43e+02  -3.8 2.92e+01  -2.5 2.27e-01 2.73e-01f  1
 654  2.7334639e+05 1.37e+00 1.39e+03  -3.8 8.39e+01  -3.0 5.95e-01 3.72e-03f  1
 655  2.7318000e+05 1.01e+00 6.35e+02  -3.8 2.12e+02  -3.5 1.00e+00 3.61e-01f  1
 656  2.7304549e+05 1.01e+00 7.21e+02  -3.8 6.39e+02  -4.0 4.92e-01 9.80e-02f  1
 657  2.7304539e+05 1.01e+00 7.21e+02  -3.8 3.32e+03  -4.4 7.12e-06 3.25e-05f  1
 658  2.7304538e+05 1.01e+00

...

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 870  2.3872514e+05 3.83e+01 6.43e+02  -3.8 1.01e+04  -4.9 4.76e-02 2.36e-02f  1
 871  2.3507673e+05 4.77e+01 9.14e+02  -3.8 2.77e+03  -4.5 3.83e-02 8.63e-01f  1
 872  2.3507661e+05 4.77e+01 9.14e+02  -3.8 1.53e+04  -5.0 8.09e-05 7.22e-06f  1
 873  2.3506946e+05 4.77e+01 1.60e+03  -3.8 4.03e+04  -5.5 2.13e-02 1.12e-04f  1
 874  2.3366512e+05 5.27e+01 7.91e+02  -3.8 1.09e+04  -5.0 1.16e-05 8.09e-02f  1
 875  2.3163836e+05 4.90e+01 1.33e+03  -3.8 4.21e+03  -4.6 1.64e-03 3.34e-01f  1
 876  2.3123123e+05 4.86e+01 1.30e+03  -3.8 3.23e+04  -5.1 1.44e-02 1.23e-02f  1
 877  2.3123108e+05 4.86e+01 1.30e+03  -3.8 5.45e+03  -4.6 1.89e-04 1.91e-05f  1
 878  2.3121942e+05 4.85e+01 1.47e+03  -3.8 1.58e+04  -5.1 1.83e-02 4.46e-04f  1
 879  2.3119636e+05 4.84e+01 1.46e+03  -3.8 5.08e+03  -4.7 1.54e-03 2.80e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 880  2.3119440e+05 4.84e+01

 982  1.6485981e+05 7.36e+01 8.24e+03  -3.8 6.67e+04  -5.1 7.41e-03 2.51e-06f  1
 983  1.6354839e+05 7.18e+01 7.80e+03  -3.8 1.35e+04  -4.7 2.16e-05 3.58e-02f  1
 984  1.6354815e+05 7.18e+01 8.03e+03  -3.8 9.20e+04  -5.1 8.85e-03 1.03e-06f  1
 985  1.6354562e+05 7.18e+01 7.92e+03  -3.8 1.21e+04  -4.7 6.73e-03 9.74e-05f  1
 986  1.6083195e+05 8.82e+01 7.80e+03  -3.8 8.92e+04  -5.2 7.34e-07 1.32e-02f  1
 987  1.6079387e+05 8.82e+01 1.68e+04  -3.8 1.96e+04  -4.8 9.97e-01 7.27e-04f  1
 988  1.5964667e+05 8.77e+01 1.68e+04  -3.8 3.31e+05  -5.2 1.37e-03 1.37e-03f  1
 989  1.5964556e+05 8.77e+01 1.64e+04  -3.8 1.97e+04  -4.8 2.02e-02 1.93e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 990  1.5752985e+05 8.55e+01 1.65e+04  -3.8 1.76e+05  -5.3 4.45e-07 3.87e-03f  1
 991  1.5669934e+05 8.48e+01 1.78e+04  -3.8 2.88e+04  -4.9 2.92e-01 1.08e-02f  1
 992  1.5669870e+05 7.93e+01 1.43e+04  -3.8 2.62e+02  -2.6 2.64e-04 6.50e-02h  1
 993  1.5651691e+05 5.60e+01

1118  4.6262560e+03 8.19e+00 1.90e+02  -3.8 1.02e+03    -  9.52e-03 1.42e-02f  1
1119  4.5902025e+03 7.55e+00 1.97e+02  -3.8 4.96e+03    -  3.41e-04 7.26e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1120  4.5901685e+03 7.55e+00 1.45e+02  -3.8 1.00e+03    -  1.16e-01 7.59e-06f  1
1121  4.0093355e+03 6.55e+00 2.02e+02  -3.8 1.13e+03    -  1.32e-04 1.31e-01f  1
1122  3.7342840e+03 6.12e+00 1.47e+02  -3.8 1.01e+03    -  3.42e-01 7.12e-02f  1
1123  1.6420285e+03 2.81e+01 6.66e+02  -3.8 1.04e+03    -  2.68e-04 5.69e-01f  1
1124  1.6402863e+03 2.81e+01 4.85e+02  -3.8 4.64e+02    -  6.92e-02 1.10e-03f  1
1125  1.5681127e+03 2.68e+01 2.16e+02  -3.8 4.71e+02    -  2.33e-01 4.58e-02f  1
1126  1.5680496e+03 2.68e+01 2.16e+02  -3.8 4.39e+02    -  1.22e-03 4.23e-05f  1
1127  1.5680335e+03 2.68e+01 2.23e+02  -3.8 4.41e+02    -  5.88e-02 1.08e-05f  1
1128  1.3172621e+03 2.30e+01 2.69e+02  -3.8 8.70e+02    -  1.36e-05 1.65e-01f  1
1129  1.2707380e+03 2.22e+01

1262  4.4005339e+01 5.82e+00 7.40e+01  -3.8 4.52e-01  -0.6 1.00e+00 5.78e-01h  1
1263  4.3947064e+01 3.58e+00 1.24e+02  -3.8 2.04e-01  -0.1 1.00e+00 4.04e-01h  1
1264  4.3818572e+01 2.20e+00 3.67e+01  -3.8 2.53e-01  -0.6 1.00e+00 3.82e-01h  1
1265  4.3450175e+01 1.28e+00 1.80e+01  -3.8 7.05e-01  -1.1 1.00e+00 3.98e-01h  1
1266  4.3090831e+01 1.04e+00 4.77e+00  -3.8 1.82e+00  -1.6 6.47e-01 1.60e-01h  1
1267  4.3087348e+01 9.81e-01 7.45e+00  -3.8 5.54e+00  -1.1 1.44e-04 4.51e-02h  1
1268  4.3065138e+01 9.56e-01 9.81e+00  -3.8 1.99e+00  -0.7 5.41e-02 2.39e-02h  1
1269  4.3001707e+01 7.74e-01 1.84e+01  -3.8 6.92e-01  -0.3 1.00e+00 1.83e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1270  4.2969815e+01 7.45e-01 5.39e+01  -3.8 2.09e+00  -0.8 6.07e-01 3.63e-02h  1
1271  4.2852013e+01 6.34e-01 2.42e+01  -3.8 7.63e-01  -1.2 9.08e-01 1.34e-01h  1
1272  4.2618136e+01 5.25e-01 2.77e+01  -3.8 1.97e+00  -1.7 2.81e-01 1.26e-01f  1
1273  4.2617300e+01 5.23e-01

1367  4.1923561e+01 9.82e-03 2.46e+00  -5.7 5.48e+01    -  9.91e-01 5.55e-01h  1
1368  4.1923518e+01 2.21e-02 2.38e+00  -5.7 3.47e+02    -  1.00e+00 8.41e-01h  1
1369  4.1923534e+01 3.13e-03 4.32e-02  -5.7 1.31e+02    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1370  4.1923540e+01 2.90e-04 2.29e-03  -5.7 3.60e+01    -  1.00e+00 1.00e+00h  1
1371  4.1923542e+01 3.46e-05 5.16e-04  -5.7 1.46e+01    -  1.00e+00 1.00e+00h  1
1372  4.1923542e+01 2.16e-08 4.15e-07  -5.7 4.04e-01    -  1.00e+00 1.00e+00h  1
1373  4.1923335e+01 5.40e-03 3.41e+01  -8.6 4.23e+02    -  2.54e-01 4.39e-01h  1
In iteration 1373, 1 Slack too small, adjusting variable bound
1374  4.1923181e+01 2.17e+00 5.72e+01  -8.6 8.29e+01    -  2.91e-01 6.10e-01h  1
In iteration 1374, 1 Slack too small, adjusting variable bound
1375  4.1923126e+01 8.77e+00 2.16e+01  -8.6 6.67e-01  -4.4 5.80e-01 5.20e-01h  1
1376  4.1923098e+01 8.80e+00 1.38e+01  -8.6 4.83e-01  -4.0 6.83e

In [9]:
solve_pyomo(para_truth[5])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 189r 1.2572854e+04 9.70e+04 7.88e+03  -3.0 4.24e+02    -  8.08e-03 9.11e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 1.2573069e+04 9.70e+04 5.16e+03  -3.0 3.82e+02    -  3.62e-01 1.09e-04f  1
 191r 1.2621707e+04 9.70e+04 4.14e+03  -3.0 3.03e+02    -  6.70e-02 3.39e-02f  1
 192r 1.3899825e+04 9.68e+04 2.25e+04  -3.0 2.94e+02    -  7.95e-05 9.04e-01f  1
 193r 1.3925501e+04 9.68e+04 1.89e+04  -3.0 1.06e+02    -  4.74e-01 4.42e-02f  1
 194r 1.3925530e+04 9.68e+04 1.84e+04  -3.0 7.80e+01    -  5.44e-02 6.65e-05f  1
 195r 1.3925801e+04 9.68e+04 1.84e+04  -3.0 5.96e+01    -  4.62e-04 2.00e-03f  1
 196r 1.3931318e+04 9.68e+04 1.75e+04  -3.0 2.77e+01    -  4.64e-02 4.08e-02f  1
 197r 1.3949347e+04 9.68e+04 1.73e+04  -3.0 4.84e+01    -  7.93e-05 6.76e-02f  1
 198r 1.4020959e+04 9.68e+04 1.46e+04  -3.0 9.89e+01    -  1.76e-01 1.48e-01f  1
 199r 1.4021033e+04 9.68e+04 1.44e+04  -3.0 8.17e+01    -  6.65e-02 1.85e-04f  1
iter    objective    inf_pr 

 499  2.2938749e+05 1.00e+02 3.17e+05  -1.0 9.74e+02  -0.9 1.82e-02 3.40e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 500  2.2940671e+05 9.92e+01 3.30e+05  -1.0 9.59e+02  -1.4 7.84e-02 1.21e-02h  1
 501  2.2941246e+05 9.89e+01 3.33e+05  -1.0 9.34e+02  -0.9 6.55e-03 3.67e-03h  1
 502  2.2942499e+05 9.79e+01 3.30e+05  -1.0 9.17e+02  -1.4 1.07e-02 9.68e-03h  1
 503  2.2942727e+05 9.78e+01 3.50e+05  -1.0 9.16e+02  -1.0 1.88e-02 1.49e-03h  1
 504  2.2943921e+05 9.67e+01 3.46e+05  -1.0 8.87e+02  -1.5 1.59e-02 1.16e-02h  1
 505  2.2944322e+05 9.63e+01 4.06e+05  -1.0 8.84e+02  -1.0 5.39e-02 3.38e-03h  1
 506  2.2945403e+05 9.52e+01 4.02e+05  -1.0 8.60e+02  -1.5 2.10e-02 1.25e-02h  1
 507  2.2946375e+05 9.43e+01 3.98e+05  -1.0 8.53e+02  -1.1 8.73e-03 9.29e-03h  1
 508  2.2946808e+05 9.38e+01 3.95e+05  -1.0 8.34e+02  -1.6 3.82e-03 5.96e-03h  1
 509  2.2946936e+05 9.37e+01 4.72e+05  -1.0 8.40e+02  -1.1 7.43e-02 1.44e-03h  1
iter    objective    inf_pr 

 616  1.6240746e+05 6.87e+01 5.40e+05  -1.0 4.12e+02  -0.6 5.61e-03 9.86e-04h  1
 617  1.6243053e+05 6.63e+01 4.61e+05  -1.0 4.98e+02  -1.1 1.74e-03 3.55e-02h  1
 618  1.6243116e+05 6.62e+01 5.91e+05  -1.0 4.00e+02  -0.7 1.68e-02 9.91e-04h  1
 619  1.6245756e+05 6.36e+01 5.60e+05  -1.0 8.34e+02  -1.1 2.27e-03 3.95e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 620  1.6245794e+05 6.36e+01 3.62e+06  -1.0 4.19e+02  -0.7 5.24e-01 6.32e-04h  1
 621  1.6246256e+05 6.31e+01 3.55e+06  -1.0 4.01e+02  -0.3 5.55e-03 7.71e-03h  1
 622  1.6249736e+05 5.93e+01 3.38e+06  -1.0 4.19e+02  -0.8 7.78e-02 6.02e-02h  1
 623  1.6249782e+05 5.92e+01 3.37e+06  -1.0 3.79e+02  -0.3 9.18e-04 8.02e-04h  1
 624  1.6250505e+05 5.86e+01 3.35e+06  -1.0 6.90e+02  -0.8 1.35e-03 1.14e-02h  1
 625r 1.6250505e+05 5.86e+01 1.00e+03   1.8 0.00e+00  -1.3 0.00e+00 2.59e-07R  6
 626r 1.6280852e+05 1.49e+01 1.43e+03   1.8 3.28e+05    -  2.25e-03 9.91e-04f  1
 627  1.6279457e+05 1.49e+01

 754  2.0936514e+04 7.35e+01 6.74e+03  -1.0 7.53e+03    -  9.34e-04 6.19e-03f  1
 755  2.0880532e+04 7.34e+01 6.69e+03  -1.0 6.84e+03    -  6.39e-03 2.00e-03f  1
 756  2.0702135e+04 7.30e+01 6.67e+03  -1.0 7.51e+03    -  3.11e-03 5.84e-03f  1
 757  2.0607598e+04 7.27e+01 6.66e+03  -1.0 9.67e+03    -  2.79e-03 3.10e-03f  1
 758  2.0250690e+04 7.19e+01 6.63e+03  -1.0 9.79e+03    -  6.10e-03 1.17e-02f  1
 759  1.9006284e+04 6.91e+01 6.59e+03  -1.0 9.17e+03    -  1.41e-02 4.32e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 760  1.8260575e+04 6.69e+01 6.22e+03  -1.0 7.48e+03    -  5.18e-02 3.39e-02f  1
 761  1.8030456e+04 6.62e+01 6.21e+03  -1.0 6.54e+03    -  3.60e-03 1.16e-02f  1
 762  1.7892164e+04 6.57e+01 6.15e+03  -1.0 6.42e+03    -  9.35e-03 6.61e-03f  1
 763  1.7676252e+04 6.51e+01 6.11e+03  -1.0 6.41e+03    -  6.71e-03 8.94e-03f  1
 764  1.7403714e+04 6.45e+01 6.08e+03  -1.0 6.51e+03    -  6.40e-03 9.42e-03f  1
 765  1.7243345e+04 6.42e+01

 878  8.2719454e+02 3.22e+01 6.33e+04  -3.8 5.54e+01  -2.6 5.73e-01 6.20e-02f  1
 879  7.1993341e+02 3.84e+01 6.24e+04  -3.8 2.06e+02  -3.0 1.40e-02 1.09e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 880  7.1989382e+02 3.84e+01 4.75e+04  -3.8 4.22e+02  -3.5 2.39e-01 7.41e-05f  1
 881  6.7740988e+02 3.69e+01 4.75e+04  -3.8 8.73e+02  -4.0 4.58e-05 4.78e-02f  1
 882  6.7717051e+02 3.69e+01 4.63e+04  -3.8 1.18e+03  -4.5 2.45e-02 2.32e-04f  1
 883  6.6899628e+02 3.68e+01 4.63e+04  -3.8 1.08e+03  -5.0 4.78e-06 2.01e-03f  1
 884  6.6882558e+02 3.68e+01 4.54e+04  -3.8 1.46e+03  -5.4 1.97e-02 1.35e-04f  1
 885  6.6158690e+02 3.68e+01 4.54e+04  -3.8 7.25e+03  -5.9 1.10e-06 3.11e-04f  1
 886  6.6145158e+02 3.68e+01 4.50e+04  -3.8 1.51e+03  -6.4 9.18e-03 1.03e-04f  1
 887  6.5223291e+02 3.67e+01 4.50e+04  -3.8 1.28e+03  -4.2 2.11e-05 1.93e-03f  1
 888  6.5214697e+02 3.67e+01 4.47e+04  -3.8 1.26e+03  -4.6 6.18e-03 7.51e-05f  1
 889  6.4745484e+02 3.67e+01

 992  3.4824928e+02 2.87e-02 2.30e+02  -3.8 1.92e+02  -3.2 1.85e-04 1.02e-04f  1
 993  3.4777782e+02 2.87e-02 2.37e+02  -3.8 6.57e+02  -3.7 3.71e-03 1.02e-03f  1
 994  3.4708282e+02 2.81e-02 2.34e+02  -3.8 3.65e+01  -2.3 9.77e-05 2.13e-02f  1
 995  3.4702478e+02 2.81e-02 2.65e+02  -3.8 7.81e+01  -2.8 4.95e-02 1.16e-03f  1
 996  3.4674418e+02 2.80e-02 2.69e+02  -3.8 1.42e+02  -2.4 1.13e-05 3.55e-03f  1
 997  3.4528975e+02 2.72e-02 2.79e+02  -3.8 8.71e+01  -2.9 5.32e-02 2.72e-02f  1
 998  3.4526702e+02 2.72e-02 2.79e+02  -3.8 3.78e+04    -  5.73e-06 2.82e-05f  1
 999  3.4337258e+02 2.69e-02 2.81e+02  -3.8 2.70e+02  -3.4 1.66e-02 1.17e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1000  3.4335394e+02 2.69e-02 2.81e+02  -3.8 3.32e+04    -  5.75e-06 2.46e-05f  1
1001  3.4168089e+02 2.68e-02 2.87e+02  -3.8 1.12e+03  -3.8 3.90e-03 2.61e-03f  1
1002  3.4166029e+02 2.68e-02 2.86e+02  -3.8 2.72e+04    -  6.52e-06 3.00e-05f  1
1003  3.3933103e+02 2.64e-02

1119  1.1523740e+02 1.92e-07 5.27e+01  -3.8 2.54e-03   1.6 1.00e+00 6.30e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1120  1.1523690e+02 1.00e-06 6.98e-02  -3.8 5.20e-03   1.1 1.00e+00 1.00e+00f  1
1121  1.1482429e+02 2.37e-05 1.18e+01  -3.8 6.81e+03    -  3.02e-04 6.34e-04f  1
1122  1.1481956e+02 9.34e-06 6.16e-02  -3.8 1.37e-02   0.7 1.00e+00 1.00e+00f  1
1123  1.1481141e+02 1.54e-05 5.84e+00  -3.8 3.21e-02   0.2 1.00e+00 6.18e-01f  1
1124  1.1480949e+02 1.29e-05 2.84e+01  -3.8 7.62e-02  -0.3 1.00e+00 1.61e-01f  1
1125  1.1480777e+02 2.11e-07 5.23e-03  -3.8 3.15e-02  -0.8 1.00e+00 1.00e+00f  1
1126  1.1480239e+02 2.16e-08 3.47e-03  -3.8 6.29e-02  -1.3 1.00e+00 1.00e+00f  1
1127  1.1478603e+02 3.89e-08 3.13e-03  -3.8 1.70e-01  -1.7 1.00e+00 1.00e+00f  1
1128  1.1475980e+02 1.30e-07 7.11e-01  -3.8 4.88e-01  -2.2 1.00e+00 5.58e-01f  1
1129  1.1472316e+02 4.01e-07 4.31e+00  -3.8 1.40e+00  -2.7 1.00e+00 2.72e-01f  1
iter    objective    inf_pr 

1248  6.2876817e+01 4.52e-06 4.18e-02  -3.8 4.67e-01  -1.9 1.00e+00 1.00e+00h  1
1249  6.2876818e+01 4.10e-04 3.39e+00  -3.8 1.56e+02    -  6.95e-01 2.01e-01h  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1250  6.2876871e+01 8.01e-07 1.03e-01  -3.8 8.02e-02  -2.4 1.00e+00 1.00e+00h  1
1251  6.2877185e+01 6.29e+00 6.24e-01  -3.8 3.73e+01    -  1.00e+00 1.00e+00h  1
1252  6.2877430e+01 2.12e-03 9.58e-02  -3.8 4.23e+01    -  1.00e+00 1.00e+00h  1
1253  6.2877435e+01 1.50e-05 3.91e-03  -3.8 3.31e+00    -  1.00e+00 1.00e+00h  1
1254  6.2877434e+01 2.04e-10 2.65e-06  -3.8 1.77e-02    -  1.00e+00 1.00e+00h  1
1255  6.2863894e+01 1.85e-03 1.21e+03  -5.7 1.30e+02    -  4.62e-01 3.81e-01f  1
1256  6.2849783e+01 1.14e+01 1.95e+03  -5.7 5.56e-01  -2.9 8.99e-01 7.00e-01h  1
1257  6.2849051e+01 1.09e+01 1.88e+03  -5.7 3.53e+02    -  1.03e-01 5.48e-02h  1
1258  6.2846258e+01 1.15e+01 1.89e+03  -5.7 1.57e+03    -  6.86e-01 1.86e-01h  1
1259  6.2845471e+01 9.97e+00

In [10]:
solve_pyomo(para_truth[6])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 149r 1.8446928e+05 5.23e+04 9.99e+02   4.7 0.00e+00  -0.8 0.00e+00 4.59e-07R  5
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 150r 1.8439937e+05 5.23e+04 1.65e+04   4.7 1.87e+07    -  4.09e-04 1.68e-06f  1
 151r 2.1375532e+05 4.47e+04 1.63e+04   2.6 3.13e+06    -  2.36e-03 2.43e-03f  1
 152  2.1376518e+05 4.47e+04 4.34e+03  -1.0 4.47e+04  -1.2 2.23e-04 2.50e-05h  1
 153  2.1380222e+05 4.47e+04 4.34e+03  -1.0 4.47e+04  -1.7 2.07e-04 1.03e-04h  1
 154  2.1381530e+05 4.47e+04 4.34e+03  -1.0 4.47e+04  -2.2 3.89e-04 4.03e-05h  1
 155  2.1387260e+05 4.47e+04 4.34e+03  -1.0 4.47e+04  -1.8 3.68e-04 1.68e-04h  1
 156r 2.1387260e+05 4.47e+04 9.99e+02   4.6 0.00e+00  -2.3 0.00e+00 4.13e-07R  3
 157r 2.1387248e+05 4.47e+04 1.66e+03   4.6 5.89e+07    -  9.36e-06 3.83e-07f  1
 158r 2.1924514e+05 4.33e+04 5.58e+03   2.6 3.82e+06    -  1.04e-02 3.51e-04f  1
 159r 2.3543039e+05 3.93e+04 5.55e+03   2.6 1.04e+07    -  2.75e-04 3.86e-04f  1
iter    objective    inf_pr 

 243  3.8468634e+05 4.24e+02 2.06e+05  -1.0 1.96e+04  -2.3 1.79e-03 2.73e-03f  1
 244  3.8465708e+05 4.23e+02 2.14e+05  -1.0 2.21e+04  -2.8 6.48e-03 1.97e-03f  1
 245  3.8464557e+05 4.23e+02 2.22e+05  -1.0 2.04e+04  -2.3 2.44e-03 8.14e-04f  1
 246  3.8464436e+05 4.23e+02 2.25e+05  -1.0 2.25e+04  -2.8 1.82e-03 8.63e-05f  1
 247  3.8462802e+05 4.22e+02 2.38e+05  -1.0 2.08e+04  -2.4 4.17e-03 1.19e-03f  1
 248  3.8457096e+05 4.21e+02 2.36e+05  -1.0 2.29e+04  -2.9 2.69e-03 3.88e-03f  1
 249  3.8455710e+05 4.20e+02 2.56e+05  -1.0 2.15e+04  -2.4 6.14e-03 1.22e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 250  3.8455619e+05 4.20e+02 2.57e+05  -1.0 2.34e+04  -2.9 1.03e-03 7.04e-05f  1
 251  3.8455219e+05 4.20e+02 2.57e+05  -1.0 1.17e+05  -3.4 8.49e-07 1.93e-04f  1
 252  3.8455072e+05 4.20e+02 2.62e+05  -1.0 2.36e+04  -3.0 4.84e-03 1.10e-04f  1
 253  3.8450466e+05 4.19e+02 2.61e+05  -1.0 2.48e+04  -3.4 2.31e-03 2.85e-03f  1
 254  3.8450374e+05 4.19e+02

 384  2.9227166e+05 3.61e+02 1.06e+04  -1.0 9.81e+03  -5.3 1.03e-02 6.35e-03f  1
 385  2.9227625e+05 3.60e+02 1.06e+04  -1.0 6.02e+03  -4.8 1.76e-03 2.02e-03h  1
 386  2.9226372e+05 3.41e+02 1.09e+04  -1.0 9.07e+03  -5.3 1.24e-03 5.35e-02f  1
 387  2.9220105e+05 3.39e+02 1.19e+04  -1.0 2.03e+04  -5.8 5.10e-02 7.52e-03f  1
 388  2.9212995e+05 3.37e+02 1.16e+04  -1.0 2.64e+04  -6.3 3.08e-04 5.31e-03f  1
 389  2.9171225e+05 3.32e+02 1.11e+04  -1.0 4.23e+04  -5.8 5.63e-03 1.75e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 390  2.9170725e+05 3.32e+02 1.48e+04  -1.0 3.68e+04  -6.3 6.55e-02 2.27e-04f  1
 391  2.9134509e+05 3.10e+02 1.38e+04  -1.0 1.37e+04  -5.9 7.23e-02 7.17e-02f  1
 392  2.9120151e+05 3.04e+02 1.31e+04  -1.0 4.50e+04  -6.4 5.95e-04 1.96e-02f  1
 393  2.9093744e+05 3.03e+02 1.28e+04  -1.0 6.49e+04  -5.9 6.77e-04 4.88e-03f  1
 394  2.9027933e+05 2.90e+02 1.20e+04  -1.0 2.03e+04  -5.5 6.43e-02 4.94e-02f  1
 395  2.9027007e+05 2.90e+02

 505  2.7666242e+05 1.17e+02 4.50e+03  -1.0 2.57e+04    -  2.31e-05 1.52e-03f  1
 506  2.7666263e+05 1.14e+02 4.19e+03  -1.0 1.81e+03  -4.5 5.76e-03 1.96e-02h  1
 507  2.7666153e+05 1.14e+02 4.16e+03  -1.0 9.29e+03  -4.9 1.42e-05 2.84e-04f  1
 508  2.7666466e+05 1.14e+02 4.14e+03  -1.0 1.29e+03  -3.6 3.41e-04 2.05e-03h  1
 509  2.7666308e+05 1.14e+02 4.12e+03  -1.0 3.29e+06    -  8.82e-09 1.04e-06f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 510  2.7666980e+05 1.13e+02 3.95e+03  -1.0 8.61e+02  -4.1 6.54e-04 7.32e-03h  1
 511  2.7666717e+05 1.13e+02 3.95e+03  -1.0 2.20e+04    -  5.26e-06 1.44e-04f  1
 512  2.7666806e+05 1.13e+02 3.95e+03  -1.0 2.86e+03  -3.7 2.05e-04 5.91e-04h  1
 513  2.7666350e+05 1.13e+02 3.95e+03  -1.0 2.23e+04    -  3.43e-06 2.50e-04f  1
 514  2.7667188e+05 1.12e+02 3.72e+03  -1.0 9.66e+02  -4.1 2.08e-03 1.03e-02h  1
 515  2.7666340e+05 1.12e+02 3.74e+03  -1.0 1.80e+04    -  6.89e-05 5.53e-04f  1
 516  2.7666395e+05 1.12e+02

...

1362  3.7917724e+01 4.33e-03 7.00e+00  -8.6 4.51e+00  -1.8 5.63e-01 2.46e-02h  1
1363  3.7850187e+01 4.31e-03 6.12e+00  -8.6 1.42e+01  -2.3 1.08e-01 3.95e-03h  1
1364  3.7752149e+01 4.24e-03 5.99e+00  -8.6 5.07e+00  -1.8 2.38e-02 1.53e-02h  1
1365  3.7615666e+01 4.01e-03 5.37e+00  -8.6 1.89e+00  -1.4 1.44e-01 5.46e-02h  1
1366  3.7568934e+01 3.98e-03 3.98e+00  -8.6 5.79e+00  -1.9 6.01e-01 6.71e-03h  1
1367  3.7530977e+01 3.93e-03 3.98e+00  -8.6 2.16e+00  -1.5 7.14e-02 1.42e-02h  1
1368  3.7376242e+01 3.85e-03 3.86e+00  -8.6 6.65e+00  -1.9 5.91e-06 2.04e-02f  1
1369  3.7360409e+01 3.83e-03 3.90e+00  -8.6 2.47e+00  -1.5 6.97e-02 5.49e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1370  3.7172944e+01 3.74e-03 3.75e+00  -8.6 7.56e+00  -2.0 9.23e-06 2.27e-02f  1
1371  3.7166468e+01 3.73e-03 4.90e+00  -8.6 2.81e+00  -1.6 4.11e-01 2.08e-03h  1
1372  3.6952883e+01 3.64e-03 4.76e+00  -8.6 8.59e+00  -2.1 6.45e-05 2.38e-02f  1
1373  3.6898442e+01 3.64e-03

1462  3.4431830e+01 1.92e-05 1.53e+00  -8.6 1.63e+00  -3.5 1.00e+00 1.00e+00h  1
1463  3.4431049e+01 2.70e-06 7.24e-01  -8.6 6.11e-01  -3.0 1.00e+00 1.00e+00h  1
1464  3.4430937e+01 2.63e-06 1.16e+02  -8.6 1.83e+00  -3.5 2.89e-01 4.45e-02h  1
1465  3.4430937e+01 2.63e-06 3.33e+01  -8.6 6.88e-01  -3.1 1.00e+00 5.17e-04h  1
1466  3.4429190e+01 1.48e-05 1.01e+01  -8.6 2.06e+00  -3.6 3.01e-01 6.74e-01h  1
1467  3.4425973e+01 5.60e-05 1.07e+02  -8.6 6.19e+00  -4.0 7.58e-01 4.13e-01h  1
1468  3.4425765e+01 5.22e-05 2.02e+02  -8.6 2.32e+00  -3.6 1.00e+00 7.09e-02h  1
1469  3.4425505e+01 4.02e-05 6.00e+01  -8.6 8.71e-01  -3.2 1.00e+00 2.36e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1470  3.4425289e+01 3.78e-05 1.49e+02  -8.6 2.61e+00  -3.7 1.00e+00 6.54e-02h  1
1471  3.4424056e+01 6.93e-06 7.23e-01  -8.6 9.80e-01  -3.2 1.00e+00 1.00e+00h  1
1472  3.4423408e+01 7.63e-06 8.41e+01  -8.6 2.94e+00  -3.7 1.00e+00 1.76e-01h  1
1473  3.4423246e+01 7.64e-06

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1570  3.4318925e+01 2.16e-06 3.49e+01  -8.6 2.78e-01  -2.7 8.87e-01 1.72e-03h  1
1571  3.4318114e+01 3.22e-06 1.74e+01  -8.6 8.35e-01  -3.2 1.00e+00 7.22e-01h  1
1572  3.4317984e+01 2.31e-06 2.81e+01  -8.6 3.13e-01  -2.7 9.62e-01 3.04e-01h  1
1573  3.4317590e+01 2.23e-06 4.41e+01  -8.6 9.39e-01  -3.2 1.00e+00 3.26e-01h  1
1574  3.4317184e+01 2.68e-06 1.95e+02  -8.6 2.82e+00  -3.7 6.18e-01 1.10e-01h  1
1575  3.4317139e+01 2.60e-06 1.63e+02  -8.6 1.06e+00  -3.3 1.00e+00 3.22e-02h  1
1576  3.4316362e+01 4.83e-06 1.31e+02  -8.6 3.17e+00  -3.7 1.51e-01 1.94e-01h  1
1577  3.4316074e+01 4.28e-06 7.80e+01  -8.6 1.19e+00  -3.3 6.80e-01 1.91e-01h  1
1578  3.4315337e+01 6.01e-06 1.26e+02  -8.6 3.57e+00  -3.8 3.66e-01 1.63e-01h  1
1579  3.4315151e+01 5.51e-06 1.53e+02  -8.6 1.34e+00  -3.4 1.00e+00 1.09e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1580  3.4314923e+01 3.80e-06

1666  3.8229608e+01 7.98e-03 4.08e+00  -8.6 4.37e+00  -1.7 2.08e-02 7.69e-03h  1
1667  3.8127001e+01 7.66e-03 3.92e+00  -8.6 1.02e+00  -1.3 1.00e+00 3.93e-02h  1
1668  3.8071246e+01 7.61e-03 3.89e+00  -8.6 4.84e+00  -1.8 1.46e-02 6.55e-03h  1
1669  3.7989132e+01 7.39e-03 3.78e+00  -8.6 1.20e+00  -1.3 1.00e+00 2.93e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1670  3.7925820e+01 7.34e-03 3.75e+00  -8.6 4.91e+00  -1.8 2.58e-02 7.24e-03h  1
1671  3.7861060e+01 7.18e-03 3.67e+00  -8.6 1.41e+00  -1.4 1.00e+00 2.15e-02h  1
1672  3.7746456e+01 7.09e-03 3.62e+00  -8.6 5.21e+00  -1.9 4.36e-02 1.27e-02h  1
1673  3.7712894e+01 7.01e-03 3.59e+00  -8.6 1.69e+00  -1.4 1.00e+00 1.05e-02h  1
1674  3.7565637e+01 6.90e-03 5.48e+00  -8.6 5.67e+00  -1.9 7.73e-02 1.57e-02h  1
1675  3.7517460e+01 6.80e-03 3.57e+00  -8.6 1.95e+00  -1.5 1.00e+00 1.43e-02h  1
1676  3.7399530e+01 6.20e-03 3.18e+00  -8.6 7.33e-01  -1.1 7.69e-01 8.94e-02h  1
1677  3.7288427e+01 6.00e-03

1765  3.4296150e+01 1.09e-02 1.53e-02  -8.6 3.85e+01  -6.5 1.00e+00 1.00e+00h  1
1766  3.4294327e+01 3.97e-02 1.11e+01  -8.6 1.16e+02  -7.0 1.00e+00 5.98e-01h  1
1767  3.4294325e+01 6.65e-02 1.00e+01  -8.6 9.52e+01  -7.5 4.91e-01 1.00e+00f  1
1768  3.4292852e+01 1.58e-01 1.39e+01  -8.6 2.86e+02  -7.9 1.00e+00 4.47e-01h  1
1769  3.4292477e+01 1.45e-01 1.39e+01  -8.6 5.64e+02  -8.4 4.58e-01 7.92e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1770  3.4292331e+01 8.55e-02 6.19e-01  -8.6 1.90e+02  -8.9 3.20e-02 5.67e-01h  1
1771  3.4292270e+01 8.24e-02 5.87e+00  -8.6 3.96e+02  -9.4 4.01e-01 3.34e-02h  1
1772  3.4292020e+01 1.69e-01 5.91e+00  -8.6 7.31e+03  -9.8 3.29e-02 1.80e-02h  1
1773  3.4291912e+01 1.68e-01 6.05e+00  -8.6 8.11e+03 -10.3 1.72e-02 5.45e-03h  1
1774  3.4291719e+01 1.68e-01 6.21e+00  -8.6 1.71e+04 -10.8 1.48e-02 4.50e-03h  1
1775  3.4291545e+01 1.67e-01 6.25e+00  -8.6 3.43e+04 -11.3 6.12e-03 2.02e-03h  1
1776  3.4291543e+01 1.66e-01

In [11]:
solve_pyomo(para_truth[7])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 353r 6.8474754e+04 8.09e+04 9.99e+02   4.9 0.00e+00   1.6 0.00e+00 2.57e-07R  5
 354r 6.8426656e+04 8.09e+04 1.16e+04   4.9 7.37e+07    -  1.42e-04 7.72e-07f  1
 355r 7.9466335e+04 7.78e+04 1.23e+04   2.8 2.83e+06    -  2.09e-03 1.09e-03f  1
 356r 9.6695606e+04 7.34e+04 1.16e+04   2.8 9.23e+06    -  1.23e-04 4.78e-04f  1
 357r 1.1028216e+05 6.99e+04 1.21e+04   2.8 1.09e+07    -  5.58e-04 3.20e-04f  1
 358r 1.1028216e+05 6.99e+04 9.99e+02   4.8 0.00e+00   1.2 0.00e+00 4.85e-07R  4
 359r 1.1027815e+05 6.99e+04 4.48e+03   4.8 5.97e+07    -  7.43e-05 4.63e-07f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 360r 1.2243495e+05 6.66e+04 4.78e+03   2.7 6.26e+06    -  7.61e-04 5.22e-04f  1
 361r 1.4369126e+05 6.11e+04 3.84e+03   2.7 1.33e+07    -  1.08e-04 4.12e-04f  1
 362  1.4370225e+05 6.11e+04 4.53e+03  -1.0 6.11e+04   0.7 2.61e-05 2.38e-05h  1
 363  1.4371236e+05 6.11e+04 4.53e+03  -1.0 6.11e+04   0.2 4.50e-05 2.32e-05h  1
 364  1.4373222e+05 6.11e+04

 478  3.7504340e+05 1.05e+02 3.78e+05  -1.0 1.56e+04  -2.8 9.91e-03 7.68e-03h  1
 479  3.7504409e+05 1.05e+02 3.90e+05  -1.0 1.54e+04  -2.4 1.87e-02 4.71e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 480  3.7504417e+05 1.05e+02 3.92e+05  -1.0 1.48e+04  -2.9 3.86e-02 1.04e-04h  1
 481  3.7504711e+05 1.04e+02 4.00e+05  -1.0 1.78e+04  -2.4 1.80e-02 8.09e-03h  1
 482  3.7504538e+05 1.03e+02 3.96e+05  -1.0 1.80e+04  -2.9 1.17e-03 8.62e-03h  1
 483  3.7501091e+05 1.02e+02 3.92e+05  -1.0 1.84e+04  -3.4 1.13e-02 1.59e-02f  1
 484  3.7501371e+05 1.01e+02 3.89e+05  -1.0 1.77e+04  -3.0 9.44e-03 6.85e-03h  1
 485  3.7501396e+05 1.01e+02 4.20e+05  -1.0 1.76e+04  -2.6 4.55e-02 3.36e-04h  1
 486  3.7501289e+05 1.01e+02 4.14e+05  -1.0 1.80e+04  -3.0 3.05e-02 4.66e-03h  1
 487  3.7501284e+05 1.01e+02 4.16e+05  -1.0 1.77e+04  -2.6 2.61e-03 6.44e-04h  1
 488  3.7501274e+05 1.00e+02 4.82e+05  -1.0 1.91e+04  -2.2 2.01e-02 1.19e-03h  1
 489  3.7500153e+05 9.86e+01

 593  3.4488001e+05 1.00e+02 1.95e+04  -1.0 1.82e+04  -5.7 2.02e-03 6.82e-03f  1
 594  3.4481538e+05 9.91e+01 1.83e+04  -1.0 1.02e+04  -5.3 3.41e-02 1.20e-02f  1
 595  3.4471901e+05 9.82e+01 1.85e+04  -1.0 2.82e+04  -5.8 8.97e-04 1.02e-02f  1
 596  3.4461674e+05 9.64e+01 1.71e+04  -1.0 1.01e+04  -5.4 4.25e-02 1.88e-02f  1
 597  3.4449073e+05 9.53e+01 1.74e+04  -1.0 3.47e+04  -5.8 5.93e-04 1.26e-02f  1
 598  3.4447468e+05 9.50e+01 1.46e+04  -1.0 9.88e+03  -5.4 6.66e-02 2.92e-03f  1
 599  3.4437650e+05 9.38e+01 1.50e+04  -1.0 3.72e+04    -  2.15e-04 1.32e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 600  3.4437611e+05 9.38e+01 1.37e+04  -1.0 2.68e+05    -  3.05e-02 1.72e-05h  1
 601  3.4392987e+05 8.82e+01 8.77e+03  -1.0 9.78e+03    -  1.79e-01 8.54e-02f  1
 602  3.4300263e+05 8.16e+01 1.19e+04  -1.0 9.21e+03    -  2.04e-03 1.60e-01f  1
 603  3.4300095e+05 8.15e+01 7.78e+03  -1.0 2.47e+04    -  1.50e-01 2.40e-04f  1
 604  3.4291186e+05 8.06e+01

 715  3.3368196e+05 2.40e-02 2.54e+02  -2.5 7.49e+00  -2.1 5.25e-01 1.00e+00f  1
 716  3.3366813e+05 2.57e-02 3.99e+01  -2.5 2.25e+01  -2.6 1.00e+00 8.62e-01f  1
 717  3.3361974e+05 5.69e-02 3.13e-01  -2.5 6.76e+01  -3.0 1.00e+00 1.00e+00f  1
 718  3.3347307e+05 5.16e-01 1.65e-01  -2.5 2.03e+02  -3.5 1.00e+00 1.00e+00f  1
 719  3.3335983e+05 7.05e-01 1.87e+01  -2.5 6.09e+02  -4.0 1.00e+00 2.60e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 720  3.3325737e+05 8.99e-01 3.76e+01  -2.5 1.83e+03  -4.5 4.21e-01 7.82e-02f  1
 721  3.3318471e+05 7.60e-01 4.31e+01  -2.5 6.86e+02  -4.1 1.00e+00 1.50e-01f  1
 722  3.3298421e+05 1.82e+00 3.18e+01  -2.5 2.06e+03  -4.5 6.59e-02 1.37e-01f  1
 723  3.3298405e+05 1.77e+00 2.73e+02  -2.5 1.69e+01  -2.3 2.03e-01 2.88e-02h  1
 724  3.3298369e+05 1.74e+00 3.60e+02  -2.5 3.33e+01  -2.8 1.00e+00 1.60e-02f  1
 725  3.3296001e+05 1.14e+00 3.96e+02  -2.5 1.05e+02  -3.3 5.29e-02 3.53e-01f  1
 726  3.3281075e+05 9.02e-01

...

3156  1.2019481e+04 2.00e+01 9.77e+03  -3.8 1.12e+03  -3.4 3.16e-05 4.90e-02f  1
3157  1.2007113e+04 1.98e+01 9.89e+03  -3.8 4.09e+02  -3.0 2.08e-02 8.55e-03f  1
3158  1.1613361e+04 1.81e+01 7.89e+03  -3.8 1.27e+03  -3.5 4.18e-03 8.92e-02f  1
3159  1.1530653e+04 1.72e+01 7.37e+03  -3.8 4.64e+02  -3.1 4.13e-02 4.99e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
3160  1.1369359e+04 1.66e+01 8.37e+03  -3.8 1.44e+03  -3.5 1.53e-01 3.18e-02f  1
3161  1.0903136e+04 1.25e+01 3.98e+03  -3.8 5.23e+02  -3.1 6.11e-02 2.49e-01f  1
3162  1.0823365e+04 1.23e+01 4.11e+03  -3.8 1.67e+03  -3.6 4.11e-02 1.36e-02f  1
3163  1.0822358e+04 1.23e+01 5.50e+03  -3.8 5.87e+02  -3.2 1.46e-01 4.82e-04f  1
3164  9.6555401e+03 1.94e+01 4.23e+03  -3.8 1.95e+03  -3.6 1.02e-02 1.77e-01f  1
3165  8.8326022e+03 1.89e+01 3.69e+03  -3.8 6.87e+02  -3.2 1.23e-01 3.35e-01f  1
3166  8.1667455e+03 1.03e+01 1.39e+04  -3.8 2.69e+02  -2.8 2.80e-04 6.91e-01f  1
3167  8.0843973e+03 1.05e+01

3269  3.4543700e+03 5.90e-02 9.19e+04  -3.8 7.78e+01  -2.3 1.21e-04 2.36e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
3270  3.4446414e+03 5.82e-02 9.04e+04  -3.8 2.08e+02  -2.7 1.00e+00 1.41e-02f  1
3271  3.3420464e+03 2.18e-01 8.64e+04  -3.8 6.30e+02  -3.2 4.20e-03 4.80e-02f  1
3272  3.1010552e+03 2.99e+00 8.71e+04  -3.8 2.93e+03  -3.7 7.79e-03 2.15e-02f  1
3273  2.9527815e+03 3.59e+00 7.99e+04  -3.8 7.72e+02  -3.3 1.06e-01 5.18e-02f  1
3274  2.7580013e+03 4.15e+00 6.31e+04  -3.8 2.70e+02  -2.8 2.57e-01 1.98e-01f  1
3275  2.7572206e+03 4.15e+00 6.01e+04  -3.8 9.98e+02  -3.3 3.28e-02 1.93e-04f  1
3276  2.6022199e+03 4.60e+00 5.18e+04  -3.8 3.13e+02  -2.9 1.47e-01 1.31e-01f  1
3277  2.6017578e+03 4.60e+00 5.00e+04  -3.8 1.34e+03  -3.4 1.26e-02 7.99e-05f  1
3278  2.1393140e+03 1.30e+01 4.28e+04  -3.8 3.55e+02  -2.9 7.02e-02 3.23e-01f  1
3279  2.0335087e+03 1.16e+01 3.49e+04  -3.8 1.46e+02  -2.5 1.84e-01 1.74e-01f  1
iter    objective    inf_pr 

3385  3.7030358e+02 9.70e+00 7.11e+04  -3.8 5.34e+02  -3.4 3.04e-05 5.74e-04f  1
3386  3.7157006e+02 9.60e+00 7.11e+04  -3.8 2.34e+01  -2.1 5.53e-02 1.09e-02h  1
3387  3.7185551e+02 9.42e+00 7.01e+04  -3.8 6.23e+01  -2.6 3.78e-05 1.73e-02h  1
3388  3.7185164e+02 9.42e+00 7.00e+04  -3.8 1.17e+02  -3.0 5.74e-03 4.36e-05h  1
3389  3.7221628e+02 9.41e+00 7.12e+04  -3.8 1.33e+01  -1.7 7.00e-02 5.03e-03h  4
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
3390  3.7221690e+02 9.47e+00 7.07e+04  -3.8 2.06e+01  -2.2 7.47e-03 8.01e-03h  2
3391  3.6767955e+02 1.66e+01 6.57e+04  -3.8 3.05e+01  -1.8 7.55e-02 4.28e-02f  1
3392  3.6709005e+02 1.66e+01 6.54e+04  -3.8 2.22e+02  -2.2 2.80e-03 6.75e-04f  1
3393  3.6481647e+02 1.66e+01 6.52e+04  -3.8 7.24e+03  -2.7 1.64e-04 7.47e-05f  1
3394  3.6019832e+02 1.66e+01 6.38e+04  -3.8 2.33e+02  -2.3 1.37e-02 5.02e-03f  1
3395  3.6020745e+02 1.66e+01 6.43e+04  -3.8 2.11e+01  -1.9 2.31e-02 8.33e-05h  1
3396  3.6235220e+02 1.62e+01

3499  1.5972310e+02 2.47e-01 1.43e+03  -3.8 2.34e+02  -4.3 3.78e-02 3.89e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
3500  1.5585615e+02 2.82e-01 7.24e+02  -3.8 4.22e+02  -4.8 2.32e-02 3.82e-02f  1
3501  1.1845018e+02 7.47e+00 1.16e+04  -3.8 5.56e+02  -5.3 7.77e-02 2.86e-01f  1
3502  1.1779934e+02 7.44e+00 1.18e+04  -3.8 9.32e+02  -5.8 2.72e-05 3.69e-03f  1
3503  1.0769353e+02 7.52e+00 5.88e+03  -3.8 5.57e+02  -6.2 1.93e-01 1.09e-01f  1
3504  1.0573029e+02 7.40e+00 6.61e+03  -3.8 7.40e+02  -6.7 4.88e-05 1.71e-02f  1
3505  9.6695783e+01 7.67e+00 7.49e+03  -3.8 1.43e+03  -7.2 6.48e-02 9.37e-02f  1
3506  9.4520553e+01 7.59e+00 6.52e+03  -3.8 4.37e+03  -7.7 3.52e-02 2.06e-02f  1
3507  9.3730476e+01 7.60e+00 5.43e+03  -3.8 2.61e+04  -8.2 1.81e-02 4.93e-03f  1
3508  9.1435635e+01 7.62e+00 5.28e+03  -3.8 9.75e+03  -7.7 1.95e-02 1.85e-02f  1
3509  9.1129235e+01 7.64e+00 5.05e+03  -3.8 6.99e+05  -8.2 2.62e-04 1.09e-04f  1
iter    objective    inf_pr 

3636  5.0730252e+01 1.95e-06 8.71e+00  -3.8 1.99e-01  -2.2 1.00e+00 5.00e-01h  2
3637  5.0730122e+01 2.10e-06 9.34e-02  -3.8 7.32e-02  -1.7 1.00e+00 1.00e+00h  1
3638  5.0729876e+01 1.14e+00 6.74e+01  -3.8 2.33e+01  -2.2 3.05e-02 6.20e-03h  2
3639  5.0729940e+01 1.04e+00 3.89e+01  -3.8 2.55e-01  -1.8 1.00e+00 2.83e-01h  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
3640  5.0722758e+01 1.10e-01 5.37e+01  -3.8 6.21e+02    -  5.05e-01 6.13e-01h  1
3641  5.0724518e+01 2.08e-05 2.01e-01  -3.8 9.82e-01  -2.3 1.00e+00 1.00e+00h  1
3642  5.0722490e+01 3.23e-02 3.80e+01  -3.8 6.28e+02    -  5.19e-01 3.96e-01h  1
3643  5.0722907e+01 6.29e-05 5.19e-02  -3.8 1.89e+00  -2.7 1.00e+00 1.00e+00h  1
3644  5.0720813e+01 2.16e+00 9.11e+01  -3.8 4.80e+02    -  4.21e-01 1.00e+00h  1
3645  5.0719772e+01 1.81e+00 6.66e+01  -3.8 1.07e+03    -  2.34e-01 1.50e-01h  1
3646  5.0720554e+01 4.36e+00 4.19e+01  -3.8 5.85e+02    -  3.80e-01 3.09e-01h  1
3647  5.0719826e+01 1.20e-01

In [12]:
solve_pyomo(para_truth[8])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 328r 7.6010036e+04 7.82e+04 9.99e+02   4.9 0.00e+00   2.1 0.00e+00 4.28e-07R  4
 329r 7.6009081e+04 7.82e+04 2.43e+04   4.9 4.50e+07    -  7.59e-05 2.46e-08f  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 330r 8.0904193e+04 7.68e+04 2.03e+04   2.8 3.23e+05    -  5.52e-04 4.39e-03f  1
 331r 8.1007392e+04 7.68e+04 2.90e+04   2.8 3.42e+05    -  5.05e-03 8.35e-05f  1
 332r 8.3327173e+04 7.62e+04 2.78e+04   2.8 3.29e+05    -  1.53e-03 1.89e-03f  1
 333r 8.4178304e+04 7.59e+04 3.00e+04   2.8 3.09e+05    -  1.36e-03 7.49e-04f  1
 334r 8.5696619e+04 7.55e+04 2.82e+04   2.8 2.86e+05    -  1.09e-03 1.46e-03f  1
 335r 8.6761247e+04 7.52e+04 2.82e+04   2.1 2.74e+05    -  1.13e-03 1.10e-03f  1
 336r 8.6931524e+04 7.52e+04 2.43e+04   2.1 1.97e+02   0.0 4.88e-02 5.89e-02f  1
 337r 8.7118834e+04 7.52e+04 2.97e+04   2.1 4.15e+02  -0.5 1.04e-01 5.10e-02f  1
 338r 8.7421326e+04 7.52e+04 2.02e+05   2.1 7.09e+01   0.9 9.93e-01 2.57e-01f  1
 339r 8.8338843e+04 7.52e+04

 486  3.8371048e+05 3.32e+02 7.75e+04  -1.0 8.49e+03  -2.1 8.78e-03 1.91e-05h  1
 487  3.8370401e+05 3.31e+02 7.97e+04  -1.0 1.16e+04  -2.5 6.13e-03 1.21e-03f  1
 488  3.8369976e+05 3.31e+02 8.53e+04  -1.0 1.07e+04  -2.1 5.06e-03 9.17e-04f  1
 489  3.8369816e+05 3.31e+02 8.90e+04  -1.0 1.31e+04  -2.6 9.22e-03 4.46e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 490  3.8369163e+05 3.30e+02 8.91e+04  -1.0 1.94e+04  -3.1 3.52e-03 1.31e-03f  1
 491  3.8369126e+05 3.30e+02 9.42e+04  -1.0 1.26e+04  -2.6 1.00e-02 8.54e-05h  1
 492  3.8368854e+05 3.30e+02 9.44e+04  -1.0 1.86e+04  -3.1 2.92e-03 6.22e-04f  1
 493  3.8368808e+05 3.30e+02 9.55e+04  -1.0 1.55e+04  -2.7 2.04e-03 9.90e-05f  1
 494  3.8368534e+05 3.30e+02 9.58e+04  -1.0 1.84e+04  -3.2 2.24e-03 4.81e-04f  1
 495  3.8367848e+05 3.29e+02 1.01e+05  -1.0 1.75e+04  -2.7 8.11e-03 1.27e-03f  1
 496  3.8366828e+05 3.29e+02 1.03e+05  -1.0 1.91e+04  -3.2 1.56e-02 1.94e-03f  1
 497  3.8366159e+05 3.28e+02

 608  3.5971405e+05 5.91e+01 2.49e+04  -1.0 1.90e+04  -4.3 4.89e-02 9.31e-02f  1
 609  3.5969525e+05 5.91e+01 2.33e+04  -1.0 2.07e+04  -4.8 1.24e-02 5.76e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 610  3.5968835e+05 5.90e+01 2.34e+04  -1.0 1.63e+04  -3.4 2.30e-03 1.18e-03f  1
 611  3.5963655e+05 5.87e+01 2.30e+04  -1.0 1.76e+04  -3.9 1.56e-02 5.15e-03f  1
 612  3.5962910e+05 5.87e+01 1.98e+04  -1.0 2.00e+04  -4.4 3.00e-02 5.44e-04f  1
 613  3.5953622e+05 5.82e+01 1.99e+04  -1.0 1.77e+04  -4.0 7.81e-04 8.12e-03f  1
 614  3.5948556e+05 5.80e+01 1.80e+04  -1.0 1.99e+04  -4.4 2.03e-02 3.73e-03f  1
 615  3.5926402e+05 5.75e+01 1.84e+04  -1.0 2.17e+04  -4.9 5.14e-03 9.53e-03f  1
 616  3.5925958e+05 5.75e+01 1.85e+04  -1.0 1.61e+04  -3.6 1.54e-03 6.21e-04f  1
 617  3.5917044e+05 5.71e+01 1.73e+04  -1.0 1.74e+04  -4.1 2.64e-02 6.29e-03f  1
 618  3.5890207e+05 5.62e+01 1.77e+04  -1.0 2.00e+04  -4.5 1.09e-02 1.72e-02f  1
 619  3.5815107e+05 5.54e+01

 728  3.3168715e+05 5.91e+01 1.66e+05  -1.0 2.04e+01   2.6 1.88e-03 7.74e-05h  1
 729  3.3168991e+05 5.89e+01 3.38e+04  -1.0 2.20e+01   2.1 5.41e-04 4.45e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 730  3.3169113e+05 5.87e+01 3.62e+04  -1.0 2.30e+01   1.6 5.65e-03 1.94e-03h  1
 731  3.3169481e+05 5.84e+01 1.59e+04  -1.0 2.46e+01   1.1 1.82e-03 5.82e-03h  1
 732  3.3170033e+05 5.79e+01 1.95e+04  -1.0 2.56e+01   0.7 1.41e-03 8.93e-03h  1
 733  3.3170037e+05 5.79e+01 5.24e+05  -1.0 2.20e+01   2.9 1.71e-03 7.59e-05h  1
 734  3.3170104e+05 5.78e+01 4.74e+05  -1.0 2.43e+01   2.4 7.18e-04 1.12e-03h  1
 735  3.3170235e+05 5.77e+01 7.33e+05  -1.0 2.49e+01   1.9 8.19e-03 2.15e-03h  1
 736  3.3170557e+05 5.74e+01 6.70e+05  -1.0 2.50e+01   1.5 1.03e-03 5.24e-03h  1
 737  3.3170558e+05 5.74e+01 1.12e+06  -1.0 2.46e+01   2.8 1.49e-03 2.40e-05h  1
 738  3.3170571e+05 5.74e+01 1.78e+06  -1.0 2.50e+01   2.3 6.39e-03 2.12e-04h  1
 739  3.3171082e+05 5.69e+01

...

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1870  2.2460596e+05 3.88e+01 8.93e+03  -3.8 5.47e+03  -4.6 1.35e-02 4.09e-02f  1
1871  2.2335064e+05 2.83e+01 6.25e+03  -3.8 1.89e+03  -4.2 2.69e-01 3.02e-01f  1
1872  2.2334984e+05 2.83e+01 6.28e+03  -3.8 6.64e+03  -4.6 1.01e-02 5.42e-05f  1
1873  2.2270131e+05 2.46e+01 5.56e+03  -3.8 2.20e+03  -4.2 3.33e-01 1.33e-01f  1
1874  2.2026682e+05 2.27e+01 4.48e+03  -3.8 7.91e+03  -4.7 1.54e-04 1.33e-01f  1
1875  2.2026611e+05 2.27e+01 4.47e+03  -3.8 2.67e+03  -4.3 8.50e-03 1.17e-04f  1
1876  2.2006306e+05 2.26e+01 4.57e+03  -3.8 9.92e+03  -4.7 4.97e-02 9.02e-03f  1
1877  2.1951483e+05 2.08e+01 4.20e+03  -3.8 3.04e+03  -4.3 3.75e-02 7.85e-02f  1
1878  2.1926643e+05 2.07e+01 4.23e+03  -3.8 1.21e+04  -4.8 2.51e-02 9.16e-03f  1
1879  2.1875065e+05 1.94e+01 3.95e+03  -3.8 3.52e+03  -4.4 1.87e-02 6.32e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1880  2.1859022e+05 1.97e+01

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1990  6.0909630e+04 6.61e+01 6.00e+03  -3.8 3.87e+04  -5.0 1.85e-02 5.59e-05f  1
1991  5.6859263e+04 6.75e+01 6.57e+03  -3.8 2.39e+05  -5.4 1.97e-03 5.37e-03f  1
1992  4.6418674e+04 7.40e+01 8.44e+03  -3.8 4.59e+04  -5.0 1.42e-02 7.23e-02f  1
1993  4.6412267e+04 7.40e+01 8.44e+03  -3.8 7.30e+05  -5.5 2.43e-07 2.74e-06f  1
1994  4.0396266e+04 7.52e+01 9.54e+03  -3.8 6.01e+04  -5.1 5.67e-03 3.06e-02f  1
1995  3.6975922e+04 7.19e+01 9.42e+03  -3.8 1.81e+04  -4.6 4.13e-02 6.00e-02f  1
1996  3.6602355e+04 7.18e+01 9.47e+03  -3.8 8.87e+04  -5.1 3.51e-04 1.30e-03f  1
1997  3.6602297e+04 7.18e+01 8.58e+03  -3.8 2.13e+04  -4.7 3.43e-02 8.60e-07f  1
1998  3.6516224e+04 7.18e+01 8.60e+03  -3.8 9.13e+04  -5.2 1.01e-06 2.85e-04f  1
1999  3.6516151e+04 7.18e+01 8.55e+03  -3.8 2.02e+04  -5.6 2.55e-03 1.68e-06f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2000  3.6509637e+04 7.18e+01

2108  2.5828727e+03 9.94e+00 1.01e+05  -3.8 1.88e+04  -4.4 1.64e-03 2.17e-04f  1
2109  2.5754001e+03 9.94e+00 1.01e+05  -3.8 3.54e+04  -4.8 2.36e-05 1.22e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2110  2.5617917e+03 9.94e+00 1.01e+05  -3.8 1.73e+04  -4.4 2.93e-03 2.82e-04f  1
2111  2.5501882e+03 9.94e+00 1.01e+05  -3.8 4.64e+04  -4.9 2.66e-05 1.46e-04f  1
2112  2.5345124e+03 9.94e+00 1.01e+05  -3.8 1.73e+04  -4.5 1.90e-06 3.46e-04f  1
2113  2.5333842e+03 9.94e+00 1.05e+05  -3.8 6.41e+04  -4.9 8.55e-03 1.05e-05f  1
2114  1.9744947e+03 1.24e+01 1.10e+05  -3.8 1.07e+04  -4.5 1.01e-01 2.65e-02f  1
2115  1.9558794e+03 1.24e+01 1.10e+05  -3.8 3.01e+04  -5.0 1.19e-05 3.89e-04f  1
2116  1.8518448e+03 1.25e+01 1.10e+05  -3.8 3.18e+04  -4.6 2.37e-06 1.27e-03f  1
2117  1.8517763e+03 1.25e+01 1.12e+05  -3.8 4.14e+04  -5.0 1.13e-02 9.86e-07f  1
2118  1.5195960e+03 1.42e+01 1.12e+05  -3.8 8.54e+03  -4.6 9.37e-02 2.12e-02f  1
2119  1.5042739e+03 1.42e+01

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2260  5.5541948e+01 2.40e-08 4.59e-02  -3.8 9.21e-03   0.2 1.00e+00 1.00e+00h  1
2261  5.5541728e+01 4.06e-09 1.64e-02  -3.8 4.07e-03   0.6 1.00e+00 1.00e+00h  1
2262  5.5540779e+01 2.44e-07 3.73e-01  -3.8 2.74e-02   0.1 1.00e+00 1.00e+00h  1
2263  5.5539485e+01 4.68e-08 6.52e-02  -3.8 1.21e-02   0.6 1.00e+00 1.00e+00h  1
2264  5.5535777e+01 3.15e-02 3.17e+01  -3.8 7.90e-02   0.1 1.00e+00 7.33e-01h  1
2265  5.5532981e+01 2.41e-02 8.81e+01  -3.8 5.65e-02  -0.4 1.00e+00 2.68e-01f  2
2266  5.5265851e+01 1.05e+00 5.19e+02  -3.8 2.00e+03    -  2.51e-02 3.22e-02f  1
2267  5.4275252e+01 2.64e+01 7.56e+03  -3.8 6.86e+02    -  1.10e-01 3.87e-01f  1
2268  5.4274399e+01 2.61e+01 5.91e+03  -3.8 3.13e+02    -  2.04e-01 1.45e-02h  1
2269  5.4267049e+01 2.36e+01 5.27e+03  -3.8 1.85e+03    -  1.08e-01 9.77e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2270  5.4232161e+01 1.25e+01

2388  5.3527449e+01 2.32e+00 5.55e+01  -5.7 2.25e-01  -3.0 1.00e+00 8.21e-01h  1
2389  5.3527417e+01 1.10e-08 3.60e+01  -5.7 1.23e-01  -2.5 1.00e+00 8.56e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2390  5.3527419e+01 1.90e-08 7.89e+00  -5.7 1.39e-01  -3.0 1.00e+00 1.00e+00h  1
2391  5.3527367e+01 4.39e-09 4.94e+00  -5.7 6.14e-02  -2.6 1.00e+00 1.00e+00h  1
2392  5.3527292e+01 1.42e+00 7.20e+02  -5.7 5.98e-01  -3.1 1.00e+00 3.21e-01h  1
2393  5.3527201e+01 1.14e+01 1.81e+02  -5.7 4.58e-01  -3.6 1.00e+00 8.81e-01h  1
2394  5.3527171e+01 9.43e+00 9.34e+02  -5.7 3.28e-01  -3.1 1.00e+00 1.61e-01h  1
2395  5.3527141e+01 5.98e+00 3.26e+02  -5.7 1.88e-01  -3.6 1.00e+00 3.10e-01h  1
2396  5.3525581e+01 7.03e+00 1.55e+02  -5.7 8.97e+03    -  9.20e-03 2.57e-02h  1
2397  5.3525483e+01 7.07e+00 1.95e+03  -5.7 5.11e+03    -  1.32e-01 2.87e-03h  1
2398  5.3525415e+01 5.73e+00 1.76e+03  -5.7 9.30e+01    -  1.06e-01 1.73e-01h  1
2399  5.3524783e+01 1.19e+01

In [13]:
solve_pyomo(para_truth[9])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 187r 6.1329410e+03 9.83e+04 6.31e+02  -4.5 2.80e+01   0.2 5.15e-01 1.01e-02f  1
 188r 6.1444335e+03 9.83e+04 6.51e+02  -4.5 8.36e+01  -0.3 1.00e+00 4.55e-02f  1
 189r 6.3242171e+03 9.82e+04 3.15e+02  -4.5 2.47e+02  -0.8 1.68e-01 2.40e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 6.9441595e+03 9.80e+04 2.12e+03  -4.5 7.05e+02  -1.3 5.36e-01 2.86e-01f  1
 191r 7.1062634e+03 9.80e+04 1.57e+03  -4.5 2.70e+02  -0.8 2.34e-03 1.96e-01f  1
 192r 7.1942252e+03 9.80e+04 3.15e+03  -4.5 7.68e+02  -1.3 2.56e-01 3.68e-02f  1
 193r 7.4901521e+03 9.79e+04 2.12e+03  -4.5 3.01e+02  -0.9 9.77e-03 3.20e-01f  1
 194r 7.5868762e+03 9.78e+04 9.86e+03  -4.5 8.46e+02  -1.4 1.00e+00 3.65e-02f  1
 195r 7.6612179e+03 9.78e+04 8.98e+03  -4.5 3.33e+02  -0.9 9.90e-02 7.24e-02f  1
 196r 7.6630778e+03 9.78e+04 1.01e+04  -4.5 9.37e+02  -1.4 4.31e-01 6.31e-04f  1
 197r 8.6673215e+03 9.75e+04 5.37e+02  -4.5 3.72e+02  -1.0 1.00e+00 8.72e-01f  1
 198r 8.6673890e+03 9.75e+04

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 500r 2.1999986e+05 2.65e+04 9.99e+02   4.4 0.00e+00   0.5 0.00e+00 3.22e-07R  4
 501r 2.1999965e+05 2.65e+04 9.99e+02   4.4 1.22e+08    -  3.06e-07 1.58e-09f  1
 502r 2.2005238e+05 2.65e+04 6.85e+04   3.0 1.50e+06    -  1.71e-03 2.61e-05f  1
 503r 2.3047119e+05 2.30e+04 7.79e+04   2.3 1.27e+06    -  4.17e-03 2.80e-03f  1
 504  2.3047669e+05 2.30e+04 4.74e+03  -1.0 2.30e+04  -0.0 1.38e-05 1.51e-05h  1
 505r 2.3047669e+05 2.30e+04 9.99e+02   4.4 0.00e+00   0.4 0.00e+00 3.02e-07R  4
 506r 2.3047610e+05 2.30e+04 3.69e+04   4.4 8.32e+07    -  4.86e-05 8.32e-09f  1
 507r 2.3536558e+05 2.13e+04 1.31e+05   3.0 1.30e+06    -  1.02e-02 1.35e-03f  1
 508r 2.3612986e+05 2.10e+04 1.75e+05   2.3 1.48e+06    -  3.19e-03 1.85e-04f  1
 509r 2.5905632e+05 1.30e+04 1.19e+05   2.3 1.55e+06    -  2.20e-03 5.36e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 510r 2.5905632e+05 1.30e+04

 628  2.9749176e+05 7.39e+01 1.48e+06  -1.0 1.15e+03  -1.9 7.81e-03 4.89e-02h  1
 629  2.9752010e+05 7.04e+01 1.44e+06  -1.0 3.22e+03  -2.4 1.77e-03 4.65e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 630  2.9752030e+05 7.04e+01 1.36e+06  -1.0 1.15e+04  -2.9 3.30e-02 1.76e-04h  1
 631  2.9752228e+05 6.82e+01 1.34e+06  -1.0 1.84e+03  -2.5 4.83e-03 3.20e-02h  1
 632  2.9752244e+05 6.82e+01 1.75e+06  -1.0 1.03e+03  -2.0 2.32e-01 6.12e-04h  1
 633  2.9752139e+05 6.47e+01 1.66e+06  -1.0 2.04e+03  -2.5 2.87e-02 1.58e-01h  1
 634  2.9752166e+05 6.46e+01 1.67e+06  -1.0 1.52e+03  -2.1 9.66e-02 2.19e-03h  1
 635  2.9752222e+05 6.43e+01 1.65e+06  -1.0 1.07e+03  -2.6 1.42e-02 3.62e-03h  1
 636  2.9752277e+05 6.13e+01 1.69e+06  -1.0 1.01e+03  -3.0 1.78e-04 4.82e-02h  1
 637  2.9752276e+05 6.12e+01 1.13e+06  -1.0 2.23e+03  -2.6 2.88e-01 3.61e-04h  1
 638  2.9752085e+05 6.10e+01 1.08e+06  -1.0 1.73e+03  -3.1 2.53e-02 4.14e-03f  1
 639  2.9752128e+05 6.07e+01

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 750  2.7290429e+05 8.58e+01 5.24e+04  -1.0 2.59e+05  -6.0 1.33e-04 8.17e-03f  1
 751  2.7286004e+05 8.57e+01 5.24e+04  -1.0 4.82e+04  -5.5 2.76e-03 2.93e-04f  1
 752  2.7209374e+05 8.49e+01 5.19e+04  -1.0 1.53e+04  -5.1 4.18e-03 1.19e-02f  1
 753  2.7207750e+05 8.49e+01 5.14e+04  -1.0 5.07e+04  -5.6 8.85e-03 2.06e-04f  1
 754  2.7063498e+05 8.31e+01 5.17e+04  -1.0 1.45e+04  -5.2 5.19e-04 2.80e-02f  1
 755  2.7061822e+05 8.31e+01 5.14e+04  -1.0 4.63e+04  -5.6 9.82e-03 1.22e-04f  1
 756  2.7051847e+05 8.30e+01 5.14e+04  -1.0 1.21e+05  -6.1 3.14e-03 7.17e-04f  1
 757  2.7025185e+05 8.25e+01 5.11e+04  -1.0 3.78e+04  -5.7 7.97e-03 6.30e-03f  1
 758  2.6925786e+05 9.61e+01 5.04e+04  -1.0 2.53e+05  -6.2 3.89e-04 4.62e-03f  1
 759  2.6905854e+05 9.61e+01 5.03e+04  -1.0 5.72e+04  -5.7 1.52e-03 3.86e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 760  2.6905678e+05 9.60e+01

...

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1610  2.1821945e+03 4.47e+01 3.69e+04  -2.5 2.20e+01   1.8 1.70e-03 1.93e-03f  1
1611  2.1822445e+03 4.55e+01 4.23e+04  -2.5 3.43e+01   1.3 1.32e-04 4.78e-03f  1
1612  2.1822513e+03 4.55e+01 4.38e+04  -2.5 1.99e+01   0.8 2.27e-02 3.50e-04h  1
1613  2.1822591e+03 4.55e+01 4.53e+04  -2.5 2.12e+01   0.3 4.33e-03 4.05e-04h  1
1614  2.1823073e+03 4.54e+01 4.63e+04  -2.5 2.56e+01  -0.2 2.94e-03 1.45e-03h  1
1615  2.1823402e+03 4.54e+01 7.27e+04  -2.5 2.16e+01   0.3 3.95e-02 1.20e-03h  1
1616  2.1833140e+03 4.62e+01 5.74e+04  -2.5 2.96e+01  -0.2 8.93e-04 1.25e-02h  1
1617  2.1833221e+03 4.62e+01 8.21e+04  -2.5 8.10e+01   0.2 2.98e-03 6.35e-05h  1
1618  2.1836664e+03 4.70e+01 1.27e+05  -2.5 2.34e+01   0.6 2.31e-02 1.14e-02h  1
1619  2.1841207e+03 4.81e+01 2.25e+04  -2.5 2.20e+01   1.1 2.81e-04 1.22e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1620  2.1841248e+03 4.81e+01

1732  1.4201820e+03 7.44e-02 1.35e+04  -2.5 6.90e+02  -3.4 1.11e-03 1.79e-02f  1
1733  1.4181641e+03 7.45e-02 1.03e+04  -2.5 1.25e+03  -3.9 3.96e-02 1.17e-03f  1
1734  1.4181643e+03 7.29e-02 3.59e+05  -2.5 6.50e-03   4.0 1.00e+00 2.10e-02f  5
1735  1.4181715e+03 2.46e-02 2.28e+05  -2.5 2.37e-03   3.5 1.00e+00 6.64e-01h  1
1736  1.4181805e+03 1.10e-02 4.20e+05  -2.5 5.13e-03   3.0 1.00e+00 5.51e-01h  1
1737  1.4181848e+03 9.79e-03 3.85e+05  -2.5 1.16e-02   2.5 1.00e+00 1.10e-01h  4
1738  1.4182416e+03 5.38e-04 1.59e+04  -2.5 1.72e-02   2.1 1.00e+00 9.32e-01h  1
1739  1.4182890e+03 4.99e-07 6.84e+01  -2.5 2.01e-02   1.6 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1740  1.4183319e+03 2.29e-06 2.46e+01  -2.5 3.41e-02   1.1 1.00e+00 1.00e+00f  1
1741  1.4182707e+03 6.41e-06 1.35e+01  -2.5 8.07e-02   0.6 1.00e+00 1.00e+00f  1
1742  1.4178962e+03 1.27e-05 1.28e+01  -2.5 2.70e-01   0.2 1.00e+00 1.00e+00f  1
1743  1.4166383e+03 2.11e-05

1852  1.0835432e+03 6.32e-02 6.61e+04  -3.8 7.51e-01   0.2 3.18e-02 2.37e-02f  1
1853  1.0835432e+03 6.15e-02 9.54e+04  -3.8 1.20e-02   3.3 2.01e-01 2.69e-02f  1
1854  1.0835423e+03 5.18e-02 1.06e+05  -3.8 8.54e-03   2.9 2.81e-01 1.58e-01f  1
1855  1.0835355e+03 3.38e-02 3.77e+04  -3.8 6.88e-03   2.4 1.82e-01 3.51e-01h  1
1856  1.0834783e+03 1.90e-03 1.69e+05  -3.8 1.20e-02   1.9 2.76e-01 1.00e+00f  1
1857  1.0834327e+03 3.00e-03 1.11e+05  -3.8 2.96e-02   1.4 3.51e-01 2.48e-01f  1
1858  1.0833333e+03 3.42e-03 7.57e+04  -3.8 8.50e-02   1.0 3.27e-01 2.03e-01f  1
1859  1.0832691e+03 3.36e-03 6.61e+04  -3.8 2.42e-01   0.5 1.61e-01 4.94e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1860  1.0832051e+03 3.34e-03 7.46e+04  -3.8 6.71e-01   0.0 1.51e-01 2.53e-02f  1
1861  1.0831512e+03 3.27e-03 9.45e+04  -3.8 1.42e+00  -0.5 1.21e-01 2.55e-02f  1
1862  1.0831160e+03 3.24e-03 2.50e+05  -3.8 2.35e+00  -1.0 6.24e-01 1.18e-02f  1
1863  6.8436066e+02 2.66e+01

1994  1.3077650e+02 1.55e+01 2.93e+03  -5.7 8.01e+00   0.8 1.24e-01 2.59e-04h  1
1995  1.3079034e+02 1.52e+01 2.91e+03  -5.7 4.57e+00    -  7.16e-03 2.09e-02h  1
1996  1.2890790e+02 1.49e+01 3.39e+03  -5.7 6.22e+02    -  1.38e-02 1.64e-01f  1
1997  1.2893223e+02 1.42e+01 1.89e+04  -5.7 1.46e-01   1.2 1.00e+00 4.63e-02h  1
1998  1.2891022e+02 1.35e+01 1.90e+04  -5.7 5.65e+01    -  1.35e-04 5.37e-02h  1
1999  1.2891068e+02 1.34e+01 5.52e+03  -5.7 1.29e-01   0.8 1.00e+00 1.15e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2000  1.2866436e+02 9.25e+00 5.93e+03  -5.7 8.06e+01    -  2.59e-03 3.12e-01h  1
2001  1.2866440e+02 9.25e+00 5.21e+03  -5.7 2.62e-01   0.3 1.45e-01 1.78e-04h  1
2002  1.2868585e+02 8.53e+00 5.03e+03  -5.7 9.20e-02  -0.2 3.28e-02 7.75e-02h  3
2003  1.2868601e+02 8.52e+00 8.27e+03  -5.7 7.89e-02   1.1 1.00e+00 6.28e-04h  1
2004  1.2875402e+02 6.24e+00 2.00e+03  -5.7 8.20e-02   0.7 1.00e+00 2.69e-01h  1
2005  1.2857869e+02 4.16e+00

2135  1.2078349e+02 3.34e+00 1.28e+04  -5.7 1.23e+00  -0.2 6.71e-03 4.43e-02h  1
2136  1.2080310e+02 3.17e+00 7.47e+03  -5.7 3.59e-01   0.2 7.55e-01 5.37e-02h  1
2137  1.2082787e+02 2.98e+00 3.53e+02  -5.7 1.28e-01   0.6 1.00e+00 5.89e-02h  1
2138  1.2086306e+02 2.73e+00 7.11e+02  -5.7 2.94e-01   0.2 1.64e-01 1.02e-01h  1
2139  1.2085008e+02 2.63e+00 2.84e+02  -5.7 2.65e+02    -  5.11e-04 4.65e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2140  1.2085009e+02 2.63e+00 2.84e+02  -5.7 5.17e+01  -0.3 8.54e-10 1.09e-06f  3
2141  1.2085711e+02 2.58e+00 4.34e+03  -5.7 1.22e-01   1.0 3.18e-01 1.91e-02h  1
2142  1.2085711e+02 2.58e+00 4.34e+03  -5.7 8.57e+02    -  4.64e-04 2.82e-06h  2
2143  1.2085683e+02 2.58e+00 4.34e+03  -5.7 4.96e+02    -  2.47e-05 3.91e-04h  1
2144  1.2085677e+02 2.58e+00 4.34e+03  -5.7 5.54e+03    -  3.91e-09 6.73e-06f  1
2145  1.2085673e+02 2.58e+00 4.34e+03  -5.7 1.74e+03    -  4.87e-08 1.04e-05f  1
2146  1.2085672e+02 2.58e+00